# 本单元用于记录LeetCode每日一题

## 问题1：[HTML 实体解析器](https://leetcode.cn/problems/html-entity-parser/description/)
### 题目
字符串替换, 将字符串中的特殊符号, 根据规则转化为指定符号。
### 分析
- 使用正则表达式进行划分, 由于我们发现待替换的字符串中都是“&xxx;”的形式，因此我们使用正则表达式进行分组
- re.split(r"(&[^&;]+;)", text):
    - [^&;]: 表示匹配不包括&和;符号
    - +: 表示前面的匹配至少出现一次
    - [^&;]+: 表示必须匹配一个非&和非;的字符
    - (): 表示将内部匹配到的东西作为一个分组整体返回(保证了分割字符中包含有&;) 
### 优化
- 直接使用replace(目标值, 更换值)进行替换即可


In [ ]:
import re
def process(text):
    word_dict = {"&quot;":"\"","&apos;":"\'","&amp;":"&", "&gt;":">","&lt;":"<","&frasl;":"/"}
    word_list = re.split(r"(&[^&;]+;)", text)
    print(word_list)
    for index, word in enumerate(word_list):
        if word in word_dict:
            word_list[index] = word_dict[word]
    return "".join(word_list)

text = "and I quote: &quot;...&quot;"
process(text)


## 问题2：[确定两个字符串是否接近](https://leetcode.cn/problems/determine-if-two-strings-are-close/description/)
### 题目
如果可以使用以下操作从一个字符串得到另一个字符串，则认为两个字符串接近:
- 交换任意两个现有字符的位置.
- 将某种字符与另一种字符全部互换。
若通过有限步的上述两种操作，可以从原字符串得到另一个字符串则认为两个字符串相近

### 分析
- 由于可以交换任意两个字符位置，因此只要保证原字符串与目标字符串的每种字符个数是一样的即可。
- 又由于可以交换任意两种字符(即把所有的a变成b,同时把所有的b变成a), 因此只用保证原字符串的中字符个数列表与目标字符串字符个数列表具有相同元素即可.(例如[1,2,1] 与 [2,1,1])

### 注意
- 由于两个步骤必须与现有的字符交换，因此如果目标字符串中有字母a，但原字符串中没有字母a那将永远无法转换。

In [ ]:
from collections import Counter

def process(word1, word2):
    ## 得到每个字母计数的数值列表
    count1 = Counter(word1)
    count2 = Counter(word2)

    if set(count1.keys()) != set(count2.keys()):
        return False
    
    count1_list = count1.values()
    count2_list = count2.values()

    ### 判断word的元素个数列表是否相同
    count = {}
    for x in count1_list:
        if x in count:
            count[x] += 1
        else:
            count[x] = 1
    for x in count2_list:
        if x not in count or count[x] == 0:
            return False
        count[x] -= 1

    # 以上代码可以通过以下方式替换(时间复杂度会上升)
    # if sorted(count1_list) != sorted(count2_list):
    #     return False

    return True

word1 = "uau"
word2 = "ssx"
process(word1, word2)


## 问题3：[最小体力消耗路径(并查集)](https://leetcode.cn/problems/path-with-minimum-effort)
### 题目
一开始你在最左上角的格子 (0, 0) ，且你希望去最右下角的格子 (rows-1, columns-1) （注意下标从 0 开始编号）。你每次可以往 上，下，左，右 四个方向之一移动，你想要找到耗费 体力 最小的一条路径。一条路径耗费的 体力值 是路径上相邻格子之间 高度差绝对值 的 最大值 决定的。

请你返回从左上角走到右下角的最小 体力消耗值 。

### 分析
- 如果使动态规划dp[i][j]表示到i,j位置的最小体力, 先从左到右遍历更新上,左位置，再用右到左遍历更新下，右位置。但实际上这种方式是不对的。因为方向的改变会导致对前面的状态改变，因此不可取。
- 如果把每个一个格子看作是一个地点，那这个题目则是让我们找最短路径，只不过这个最短路径的定义不同。我们可以使用并查集方式，每次放入一个点，当我们加入一条权值为 x 的边之后，如果左上角和右下角从非连通状态变为连通状态，那么 x即为答案(我们先根据权重从小到大排列)。

### 注意
- 联通性判断类型的题目都可以使用并查集


In [ ]:
class UnionFind:
    """"标准并查集对象
    """
    def __init__(self, n: int):
        self.pre = list(range(n))  ## 初始化每一个点都是以自己为主元

    ## 查找函数
    def find(self, x):
        if self.pre[x] == x:
            return x
        else:
            self.pre[x] = self.find(self.pre[x])
            return self.pre[x]

    ## 并函数
    def join(self, x, y):
        fx = self.find(x)
        fy = self.find(y)
        if fx != fy:
            self.pre[fx] = fy # 定义fx的上级为fy

    ## 连通性判断x,y是否联通
    def connected(self, x: int, y: int) -> bool:
        x, y = self.find(x), self.find(y)
        return x == y


def process(heights):
    m, n = len(heights), len(heights[0])
    edges = []  # 构造边对象
    for i in range(m):
        for j in range(n):
            iden = i * n + j # 当前的坐标拉平后的位置
            if i > 0:  ## 由于是双向关系，因此只用保证上下，左右关系即可
                edges.append((iden - n, iden, abs(heights[i][j] - heights[i - 1][j])))
            if j > 0:
                edges.append((iden - 1, iden, abs(heights[i][j] - heights[i][j - 1])))
    edges = sorted(edges, key=lambda x: x[2])
    uf = UnionFind(m * n)  # 初始化并查集
    ans = 0

    for x, y, v in edges:
        uf.join(x, y)
        if uf.connected(0, m * n - 1):
            ans = v
            break
    return ans

## 问题4：[猜数字游戏](https://leetcode.cn/problems/bulls-and-cows/)
### 题目
给你一个秘密数字 secret 和朋友猜测的数字 guess ，请你返回对朋友这次猜测的提示。提示的格式为 "xAyB" ，x 是公牛个数， y 是奶牛个数，A 表示公牛，B 表示奶牛。请注意秘密数字和朋友猜测的数字都可能含有重复数字。
- 猜测数字中有多少位属于数字和确切位置都猜对了（称为 "Bulls"，公牛），
- 有多少位属于数字猜对了但是位置不对（称为 "Cows"，奶牛）。也就是说，这次猜测中有多少位非公牛数字可以通过重新排列转换成公牛数字。

### 分析
- 维护两个数组true_num和guess_num, true_num[i]表示数字i在secret中出现的次数，guess_num[i]表示数字i在guess中出现的次数
- 同时遍历secret 和 guess的i位置, 如果secret[i] == guess[i]则公牛数字加1, 否则
    - 先进行判断, 当前的正确数字是否已经出现在了guess_num中, 如果是则奶牛数字加1, 同时guess_num对应位置数字出现次数减一
    - 当前的正确数字还没出现在guess_num中，则true_num对应位置+1
    - 再判断猜测数字，是否已经出现在了true_num中, 如果是则奶牛数字加1, 同时true_num对应位置数字出现次数减一
    - 当前的猜测数字还没出现在true_num中，则guess_num对应位置+1


In [ ]:
def process(secret, guess):
    true_nums = [0 for _ in range(10)]
    guess_nums = [0 for _ in range(10)]
    bulls = 0   # 公牛结果
    cows = 0    # 奶牛结果
    for true_num, guess_num in  zip(secret, guess):
        # 判断是否为公牛数字
        if true_num == guess_num:
            bulls += 1
            continue
        
        # 判断是否为奶牛数字
        if guess_nums[int(true_num)] != 0:
            cows += 1
            guess_nums[int(true_num)] -= 1
        else:
            true_nums[int(true_num)] += 1
        
        # 继续判断是否为奶牛数字
        if true_nums[int(guess_num)] != 0:
            cows += 1
            true_nums[int(guess_num)] -= 1
        else:
            guess_nums[int(guess_num)] += 1
        
    return f"{bulls}A{cows}B"

secret = "1123"
guess = "0111"
process(secret, guess)

## 问题5：[需要添加的硬币的最小数量](https://leetcode.cn/problems/minimum-number-of-coins-to-be-added/description/)
### 题目
给你整数数组coins, 表示可用的硬币的面值, 以及一个整数target。如果存在某个coins的子序列总和为x，那么整数x就是一个可取得的金额。我们需要往coins中添加数字, 使得对于[1, target]的每一个值都是可取得的(即,都等于coins中某一个子序列的总和)。请返回最少需要添加多少个数字。

### 分析
- 我们需要将coins进行排序后遍历, 从最小值开始, 如果我们可以得到[1, x]中任意数都可以取得。
    - 此时如果下一个硬币比x大且不是x+1这个数值, 那我们将永远无法得到x+1(因为后面的coin都比x+1大, 意味着这个x+1是无法被取得), 因此我们必须添加x+1这个硬币。当我们添加了x+1这个硬币, 此时我们可以得到的取值范围就变成了[x+1, x+x+1], 且[1, x]我们依然可以取得到, 因此添加完x+1这个硬币后, 我们可以取得的范围为[1, x+x+1]
    - 如果这个值比x小(假设为s), 意味着我们可以不用添加硬币, 而还可以扩大取值范围依到[1, x+s]
- 一直循环直到我们的范围覆盖了[1, target]
- 有可能我们循环了完了coins都没能达到target, 我们还得继续添加, 因此使用了一个技巧, 在while中使用index来控制是否前进数组。如果我们添加硬币, 则还需要判定当前的coin, 因此我们不用增加index, 从而实现重复判定。


In [ ]:
def process(coins, target):
    coins.sort()
    ans = 0
    max_coin = 0  # 初始为0
    index = 0
    while max_coin < target:
        if index < len(coins) and coins[index] <= max_coin+1:   # 如果小于等于最大范围+1, 我们都可以不用添加硬币, 从而增大范围
            max_coin += coins[index]
            index += 1
        else:          # 如果当前硬币大于了最大范围, 我们需要添加一个max_coin+1元素
            ans += 1  
            max_coin = max_coin*2 + 1   # 注意我们此时并不增加index, 继续让其判定当前的coin

    return ans

coins = [1,4,10]
target = 19
process(coins, target)

## 问题6：[访问完所有房间的第一天](https://leetcode.cn/problems/first-day-where-you-have-been-in-all-the-rooms/description/)
### 题目
给你一个长度为n数组nextVisit(访问房间号)。在接下来的几天中，你访问房间的次序将根据下面的规则决定：假设某一天，你访问i号房间。
- 如果算上本次访问，访问 i 号房间的次数为**奇数**，那么第二天需要访问第nextVisit[i]号房间 (数据会保证 0<= nextVisit[i] <= i)。
- 如果算上本次访问，访问 i 号房间的次数为**偶数**，那么第二需要访问(i + 1) mod n 号房间 (访问下一间, 如果i为最后一间, 则从头访问第0间)。

请返回你访问完所有房间日期编号(也就是多少天后, 你第一次把所有的房间都访问完).

### 分析
- 注意到关键点 nextVisit[i] <= i, 意味着当奇数次访问第i号房间时, 一定会回访前面的房间。因此当我访问到第i号房间时, 对于i左边的房间, 我们一定都访问了偶数次(不然不可能到达i, 可以反证)。
- 当我们从i号房间需要回访到j号房间时, 此时[j, i−1]范围内的房间都处于访问偶数次的状态。那么当我们访问这个范围内的每个房间时，算上本次访问，访问次数一定是奇数。所以要想重新回到i, 对于[j, i−1]范围内的每个房间，我们都需要执行一次「回访」。
- f[i] 表示从 「访问到房间i且次数为奇数」 到 「访问到房间i且次数为偶数」 所需要的天数。这个天数是固定的, 只有经历过这个天数后, 才能访问到下一间i+1号房间。注意: 首次访问房间i的一天，和重新回到房间i的一天, 因此状态转移时+2天
- 状态转移方程:f[i] = 2 + sum(f[j], f[2]...f[i-1]), 初始状态f[0]=0, 这里的j为nextVisit[i]
- 我们最后的答案是: sum(f[0], f[1], ..., f[n-1]) + 1; 这里加1是访问第n间的那天
- 由于我们这里需要多次求和, 因此我们定义f[n]的前n-1项和（前缀和）s[n] = sum(f[0], f[1], ..., f[n-1]), 状态转移方程就变为了: f[n] = s[n] - s[n-1] + 2。 我们的最终结果即求: s[n] + 1
- 最后的转移方程为: s[i+1] = s[i] * 2 - s[j] + 2

In [ ]:
def process(nextVisit):
    s = [0] * len(nextVisit)
    for index, num in enumerate(nextVisit[:-1]):
        s[index+1] = s[index] * 2 - s[num] + 2
    return s[-1]+1-1    # 我们求的是日期, 我们从第0天开始的(第0天的访问算天数的), s[-1]+1是经过了多少天

nextVisit = [0,0]
process(nextVisit)

## 问题7：[验证二叉树的前序序列化](https://leetcode.cn/problems/verify-preorder-serialization-of-a-binary-tree/description/)
### 题目
给定一串以逗号分隔的序列("9,3,4,#,#,1,#,#,2,#,6,#,#")，验证它是否是正确的二叉树的**前序序列化**。编写一个在不重构树的条件下的可行算法。注意不允许重建树

### 分析
- 我们知道在树（甚至图）中，所有节点的入度(该节点有几个父节点)之和等于出度(该节点有几个子节点)之和。
- 每个非#节点, 会提供总入度+1, 总出度+2; 每个#节点, 会提供总入度+1, 总出度+0; 注意根节点只提供2个出度
- 遍历整个序列, 如果总入度和总出度的差小于0则说明该二叉数无效。注意在遍历中如果再加入某个节点前已经出现了入度 == 出度, 说明不可能再能加入新的节点(因为入度==出度意味着每个叶节点都是null了)

### 方案二: 模拟递归栈
- 把有效的叶子节点使用 "#" 代替。 比如把 4## 替换成 # 。此时，叶子节点会变成空节点！
- 先把节点压入栈中, 检查栈顶元素是否满(数字 # #)的结构, 如果时则抛出栈顶三元素, 把#压入栈顶(即把 4## 替换成 #), 循环上述操作, 直到无法压缩, 再遍历下一个节点。
- 最后判断是否只剩下["#"]即可

In [ ]:
def process(preorder):
    nodes = preorder.split(",")

    if nodes[0] == "#" and len(nodes) > 1:  # 根节点为空, 直接返回
        return False

    rudu = -1    # 根节点本是不增加入度的
    chudu = 0    
    for node in nodes:
        if rudu == chudu:   # 如果入度等于出度, 说明在进入这个节点之前, 已经达到了是完整二叉树了, 不能再加节点了
            return False
        
        rudu += 1           # 无论是否为空节点, 都会增加入度
        if node != "#":
            chudu += 2
        
    return chudu-rudu == 0

preorder = "7,2,#,2,#,#,#,6,#"
process(preorder)

In [ ]:
# 方案二使用栈
def process(preorder):
    stack = []
    for node in preorder.split(','):
        stack.append(node)
        while len(stack) >= 3 and stack[-1] == stack[-2] == '#' and stack[-3] != '#':
            stack.pop(), stack.pop(), stack.pop()
            stack.append('#')
    return len(stack) == 1 and stack.pop() == '#'


## 问题8：[统计将重叠区间合并成组的方案数](https://leetcode.cn/problems/count-ways-to-group-overlapping-ranges/description/)
### 题目
给你一个二维整数数组ranges，其中ranges[i] = [start, end]表示 start 到 end之间(包括两端点)的所有整数都包含在第i个区间中。请返回将ranges分为两个组, 满足以下规则: 
- 如果ranges[i]与ranges[j]有重叠, 则两个区间必须在一个组内。
- 每个ranges[i]只能在一个组中。
- 组可以没有元素

请返回 组的方案数(有多少中划分)

### 分析
- 我们可以将区间进行整合，看能得到多少个互不重叠的整合区间。先将区间排序, 并遍历每个区间, 如果当前区间和**整合区间**有重叠, 则更新整合区间左右边界(只用更新右边界), 如果没有重叠, 则说明当前区间为下一个整合区间的起始区间。
- 遍历结束后, 返回整和区间个数, 分为两组的情况数为(2**整合区间个数)


In [ ]:
def process(ranges):
    ranges.sort(key=lambda x: x[0])
    left = ranges[0][0]
    right = ranges[0][1]
    count = 1
    for x, y in ranges:
        print(x,y)
        if left <= x <= right:
            right = max(right, y)  # 合并区间
        else:
            count += 1  # 整合区间数 +1
            left = x    # 重置整合区间的左右边界
            right = y
    return pow(2, count, 1_000_000_007)

ranges = [[6,10],[5,15]] 
process(ranges)

## 问题9：[所有可能的真二叉树](https://leetcode.cn/problems/all-possible-full-binary-trees/description/)
### 题目
给你一个整数n, 请你找出所有可能含n个节点的真二叉树(树中每个节点的值为0). 返回的结果是每种情况的根节点(即List[Node])。

真二叉树定义:树中每个节点恰好有 0 或 2 个子节点

### 分析
- 每个结点的子结点数是0或2，此时可以推出真二叉树中的结点数n为奇数(由归纳法证明,每次加节点只能2个2个加), 因此当n为偶数时直接返回null
- 真二叉树的左右子树也一定是真二叉树, 因此我们可以采用分治法。对于节点为n的真二叉树的所有情况我们是可以得到的, 因此只要我们知道左右树的节点个数, 即可组合得到所有可能的真二叉树。假设左子树的数目为i, 则右子树的节点数目则为 n−1-i (因为左右子数结点和为n-1)，则我们可以推出左子树与右子树的节点数目序列为：$$[(1,n−2),(3,n−4),(5,n−6),⋯,(n−2,1)]$$
- 当 n=1时，此时只有一个结点的二叉树是真二叉树；当 n>1时，分别枚举左子树和右子树的根结点数，然后递归地构造左子树和右子树，并返回左子树与右子树的根节点列表。确定左子树与右子树的根节点列表后，分别枚举不同的左子树的根节点与右子树的根节点，从而可以构造出真二叉树的根节点。


In [ ]:
from functools import cache
from typing import List, Optional


class TreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right
        
@cache
def process(n) -> List[Optional[TreeNode]]:
    ans = []
    if n % 2 == 0:  # 如果为偶数则直接返回
        return ans
    if n == 1:  # 如果为1，则直接返回根节点
        ans.append(TreeNode(0))
        return ans
    # 分别递归左右子树, 左右子树的节点均为奇数
    for left_node in range(1, n-1, 2):
        left_ans = process(left_node)
        right_ans = process(n-1-left_node)
        # 将得到的左右子数的情况进行组合
        for left_node in left_ans:
            for right_node in right_ans:
                root = TreeNode(0, left_node, right_node)  # 这是一种组合
                ans.append(root)
    return ans
    


## 问题10：[使数组连续的最少操作数](https://leetcode.cn/problems/minimum-number-of-operations-to-make-array-continuous/description/)
### 题目
给你一个整数数组nums。每一次操作中，你可以将nums中任意一个元素替换成任意整数。请你返回使nums连续的最少操作次数。

数组连续定义:
- nums 中所有元素都是 互不相同 的。
- nums 中 最大 元素与 最小 元素的差等于 nums.length - 1 。

### 分析
- 正难则反, 我们可以计算出“最多有多少个元素不用改变”, 该问题其实就是在问固定长度的滑动空间(长度为nums.length - 1), 包含的最多点有多少个
- 我们首先将nums去重后排序(假设得到的数组为a), 使用滑动窗口模板, 先滑动右端点, 如果达到收缩条件(左右端点的差大于了nums.length - 1), 由于求最大，因此在收缩时记录答案


In [ ]:
def process(nums):
    n = len(nums)
    a = sorted(set(nums))
    ans = 0
    left = right = 0
    while right < len(a):  
        right += 1                    # 右边界右移
        # 此时窗口中的数值为[left, right)
        ans = max(ans, right-left)
        while right < len(a) and a[right] > a[left]+n-1: # 满足收缩条件
            left += 1
    
    return n-ans

nums = [1,2,3,10,20]
process(nums)

## 问题11：[树节点的第 K 个祖先](https://leetcode.cn/problems/kth-ancestor-of-a-tree-node/description/)
### 题目
给你一棵树，树上有n个节点，按从0到n-1编号。树以父节点数组的形式给出，其中parent[i]是节点i的父节点。树的根节点是编号为 0 的节点。树节点的第 k 个祖先节点是从该节点到根节点路径上的第 k 个节点。

实现 TreeAncestor 类:
- TreeAncestor(int n， int[] parent)对树和父数组中的节点数初始化对象。
- getKthAncestor(int node, int k) 返回节点 node 的第 k 个祖先节点。如果不存在这样的祖先节点，返回 -1 。

### 分析


## 问题12：[节点与其祖先之间的最大差值](https://leetcode.cn/problems/maximum-difference-between-node-and-ancestor/description/)
### 题目


### 分析

## 问题13：[有向无环图中一个节点的所有祖先](https://leetcode.cn/problems/all-ancestors-of-a-node-in-a-directed-acyclic-graph/description/)
### 题目
给你一个正整数n，它表示一个 有向无环图 中节点的数目，节点编号为[0, n-1]. 一个二维整数数组edges,其中edges[i] = [from, to] 表示图中一条从from到to的单向边(从节点from到节点to)。请你返回一个数组 answer，其中 answer[i]是第i个节点的所有祖先 ，这些祖先节点升序排序。

祖先定义: 如果 u 通过一系列边，能够到达 v ，那么我们称节点 u 是节点 v 的 祖先 节点。
### 分析
- 在遍历边时反向保存, key为目标节点, value为源节点集合, 最后得到的图为 {3:[1,2,4]} 表示3的父节节点为1,2,4
- 不断将源节点加入到目标节点的祖先集合中, 直到遍历结束(由于无环图, 因从总会遍历结束, 我们使用一个集合visited来保存遍历过的节点, 如果遍历过就跳过, 避免重复遍历)
- 我们使用while循环+队列的方式, 将每一个节点的父节点放入队列中, 这样可以遍历到父节点的父节点, 直到遍历完成

In [ ]:
import bisect
def process(n: int, edges):
    graph = [[] for _ in range(n)]
    for edge in edges:
        graph[edge[1]].append(edge[0])  # 构建图

    ans = [[] for _ in range(n)]

    for i in range(n):  # 遍历每个节点
        q = [i]         # 队列
        visited = [False] * n   # 保存遍历过的节点
        visited[i] = True
        while q:
            node = q.pop(0)
            for neighbor in graph[node]:
                if not visited[neighbor]:   # 如果这个父节点没有遍历过
                    visited[neighbor] = True
                    bisect.insort(ans[i], neighbor)  # 记录到答案中
                    q.append(neighbor)
    return ans

n = 8
edgeList = [[0,3],[0,4],[1,3],[2,4],[2,7],[3,5],[3,6],[3,7],[4,6]]
process(n, edgeList)

## 问题14：[修改后的最大二进制字符串](https://leetcode.cn/problems/maximum-binary-string-after-change/description)
### 题目
给你一个二进制字符串 binary ，它仅有 0 或者 1 组成。你可以使用下面的操作任意次对它进行修改：
- 操作 1: 如果二进制串包含子字符串 "00" ，你可以用 "10" 将其替换。比如， "00010" -> "10010"
- 操作 2: 如果二进制串包含子字符串 "10" ，你可以用 "01" 将其替换。比如， "00010" -> "00001"

请你返回执行上述操作任意次以后能得到的 最大二进制字符串。

### 分析
- 从操作一和操作二可以得出, 我们一直可以把0往左移动, 直到碰到00之后, 将其转化为10, 从而使得二进制字符增大。从这个角度来看, 最左边的0是无论无何都不可以被移动的, 而其他位置的0都可以向右移动, 与最左侧的0配合形成00->10的转换。
- 最终的最大二进制字符串就是除了最左边的0之外, 其他位置均为1(这种说法不对, 由于字符串可能以多个000开头, 此时都可以将0000->1110的形式), 因此连续的0开头的字符串, 我们应该找到最末尾处的那个0的位置
- 其实换个思路, 我们可以找到首个0的位置, 只经过步骤二即可将该位置后的所有1都压缩到尾部, 即为1111000000111111, 此时我们可以直到最终答案的那个0，就是在压缩尾部的1的左边
- 注意全为1的字符串和只有最后一位为0(最左侧的0刚好是最后一个位置)的字符串不用移动, 直接返回即可

### 技巧
- str.find(sub)返回子字符串sub在字符串str中第一次出现的位置, 如果没有找到, 返回-1
- str.count(sub, start, end)返回子字符串sub在字符串str中从start到end出现的次数, 如果没有找到, 返回0

In [ ]:
def process(binary):
    index_0 = binary.find('0')  # 找到最左侧的0的位置
    if index_0 == -1 or index_0 == len(binary) - 1 :
        return binary
    count_1 = binary.count('1', index_0)  # 统计首个0位置的右边有多少个1
    # 最终答案为
    ans = "1"*(len(binary)-1- count_1) + "0" + "1"*count_1

    return ans
    
process("01")

## 问题15：[互质树](https://leetcode.cn/problems/tree-of-coprimes/description/)
### 题目
给你一个n个节点的树（也就是一个无环连通无向图），节点编号从0到n-1，且恰好有n-1条边，每个节点有一个值。树的根节点为0号点。nums[i]表示第 i个节点的值(节点值可以重复), edges[j]=[u, v]表示节点u和节点v在树中有一条边。

无向图中祖先的定义: 从节点i到根最短路径上的点都是节点i的祖先节点。一个节点不是它自己的祖先节点。

请你返回一个大小为 n 的数组 ans ，其中 ans[i]是离节点i最近的祖先节点且满足nums[i]和nums[ans[i]]是互质的，如果不存在这样的祖先节点，ans[i]为-1。

### 分析
- 最暴力的做法是，枚举x的所有祖先节点(可以使用广度优先搜索), 从最近祖先开始, 直到找到和x互质的祖先. 但这种方式如果树刚好为一条链子,那么对于每个x,枚举其父节点复杂度是O(n), 因此所有节点遍历完就是O(n^2)
- 由于我们直到每个节点值介于[1,50]之间, 我们可以提前构造出一个映射, 用于保存在[1,50]范围内, 从1开始的每个数字的互质列表, coprime[i]表示与数值i互质的所有数(这些数值在1,50范围)
- 我们使用val_depth_id数组保存每个数值对应的节点信息，即val_depth_id[i]: 数值i为的节点的(层深度, 节点号), 由于存在数值重复, 我们需要使用回溯法, 恢复到上层同数值节点的信息。
- 在深度遍历时, 我们对每个节点, 都可以保存下他数值所对应的(层深度, 节点号), 每次更新val_depth_id[value]时, 都会使得该value对应的层数+1

In [ ]:
import math
# 互质匹配表
coprime = [[j for j in range(1, 51) if math.gcd(i, j) == 1]
           for i in range(51)]

def process(nums, edges):
    n = len(nums)
    ans = [-1] * n
    graph = [[] for _ in range(n)]  # 构造树
    for x, y in edges:
        graph[x].append(y)
        graph[y].append(x)
    
    val_depth_id = [(-1, -1)] * 51  # 保存数值对应的节点信息

    # 深度优先遍历, 当前节点为x, 父节点为parent, x的节点深度为depth
    def dfs(x, parent, depth):
        value = nums[x]  # 获得当前节点的值

        # 根据节点值, 找到与节点值互质的数值, 并找到这些数值对应的节点信息
        # 这里通过max函数找到最大深度, 也就是最临近的
        ans[x] = max([val_depth_id[j] for j in coprime[value]])[1]

        # 保存回溯值
        tmp = val_depth_id[value]
        # 更新val_depth_id, 将当前节点的层数和节点号进行更新
        val_depth_id[value] = (depth, x)
        
        for child in graph[x]:
            if child == parent:  # 跳过父节点
                continue
            dfs(child, x, depth+1)
        
        # 回溯上层的val_depth_id状态
        val_depth_id[value] = tmp
    
    dfs(0, -1, 0)  # 从根节点开始遍历
    return ans
    


## 问题16：[找到冠军 I](https://leetcode.cn/problems/find-champion-i/description/)
### 题目
给你一个下标从0开始、大小为n * n的二维布尔矩阵grid 。如果 grid[i][j] == 1，那么i队比j队强；否则j队比i队强。(i != j)。请返回出最强的队。要求复杂度为O(n)

### 分析
- 我们可以遍历每一个grid[i], 如果其中的比分只有一个为0(即只有自己grid[i][i]=0), 说明这个i比其他人都要强，但这种算法复杂度为O(n^2)
- 我们可以使用打擂台的方式, 假设最强队为ans=0, 我们从1开始遍历, 遍历grid[i][ans] = 1, 说明i比目前的ans强, 我们更新ans=i, 否则继续考察grid[i+1][ans], 即ans一定是前[0, i] 最强的选手。

In [ ]:
def process(grid):
    ans = 0
    for index, group in enumerate(grid):
        print(index)
        if group[ans] == 1:  # 比较index和ans谁比较强
            ans = index    # 更新当前最强者
    return ans

process([[0,1],[0,0]])

## 问题17：[找到冠军 II](https://leetcode.cn/problems/find-champion-ii/description/)
### 题目
一场比赛中共有 n 支队伍，按从 0 到  n - 1 编号, 给你一个整数 n 和一个下标从 0 开始、长度为 m 的二维整数数组 edges 表示这个有向无环图，其中 edges[i] = [u, v] 表示图中存在一条从 u 队到 v 队的有向边。表示队伍u强于队伍v。如果这场比赛存在**唯一**一个冠军, 则返回将会成为冠军的队伍。否则，返回 -1 (可能存在a队与b队没有进行过比赛)

### 分析
- 使用淘汰赛机制, 开始时ans = [0, n-1], 遍历每条边当出现[u, v]时, 在ans中去掉v, 如果最后剩下的ans只有一个则为最强者, 如果有多个则返回-1

In [ ]:
def process(n, edges):
    ans = set(range(n))
    for u, v in edges:
        if v in ans:
            ans.remove(v)
    
    return ans.pop() if len(ans) == 1 else -1


## 问题18：[尽量减少恶意软件的传播](https://leetcode.cn/problems/minimize-malware-spread/description/)
### 题目
由 n 个节点组成的网络，用 n × n 个邻接矩阵图 graph 表示。在节点网络中，当 graph[i][j] = 1 时，表示节点i能够直接连接到另一个节点 j(graph[i][j]=graph[j][i], 且graph[i][i]=1).一些节点 initial 最初被恶意软件感染。只要两个节点直接连接，且其中至少一个节点受到恶意软件的感染，那么两个节点都将被恶意软件感染。这种恶意软件的传播将继续，直到没有更多的节点可以被这种方式感染。

请从 initial 中移除一个节点能够使得最终的受感染节点数最少, 返回该节点。(如果有多个节点满足条件，就返回索引最小的节点)。

### 分析
- 找到受感染节点中的最大联通块。(类似于统计树中合法路径数目, 遍历每个节点, 如果当前节点为质数, 则将当前连通块的节点个数加到结果中, 最终得到的结果就是“不包含质数的连通块”集合)
- 遍历每个节点, 如果当前节点为污染节点, 则将当前连通块的节点个数进行记录. 使用一个size数组来记录每个非污染连通块的大小, 例如size[1]表示包含1这个节点的污染连通块的大小。
- 而如果连通块内至少有两个节点被感染，无论移除哪个节点，仍然会导致连通块被感染.所以我们要找的是**只包含一个**被感染节点的连通块，并且这个连通块越大越好


In [ ]:

def process(graph, initial):
    n = len(graph)
    initial = set(initial)

    # 得到邻接图, G[i]表示与i节点有边的其他节点
    Graph = [[] for _ in range(n+1)]  
    for i in range(n-1):
        for j in range(i+1, n):
            if graph[i][j] == 1:
                Graph[i].append(j)
                Graph[j].append(i)
    print("邻接图: ", Graph)

    # 计算以node为根节点的连通块大小. 如果该连通块中又出现了一个受污染节点, 那么我们将标识置为false(意味着当前连通块中还存在至少一个污染节点)
    def dfs(node, father):
        nonlocal size, flag, vis
        size += 1
        vis[node] = True           # 不能根据flag提前返回, 需要将所有的这个block块中节点标记为 已经访问
        if node in initial:        # 如果当前节点为污染节点, 则说明当前块中已经有2个污染节点了
            flag = False
        for child in Graph[node]:  # 遍历node的所有邻居
            if vis[child] or child == father:  # 如果子节点已经被访问过, 或遍历到父节点了, 则跳过
                continue
            if child in initial:   # 如果为污染节点则标识为false
                flag = False
            dfs(child, father)
    
    vis = [False] * n   # 记录访问情况, false为未访问
    ans_index = -1      # 需要删除的节点index
    max_nodes = 0       # 受感染最大联通块中的节点数量
    for x in initial:   # 遍历每个污染节点
        if vis[x]:      # 如果该节点已经被访问过, 说明该节点已经被计算过
            continue
        size = 1
        flag = True
        # 统计x(受污染的节点)的所有邻居
        for y in Graph[x]:
            if not vis[y]:       # y没有被计算过
                # 拿到y为根的联通块
                dfs(y, x)
        
        if size > max_nodes and flag==True:  # 更新只包含一个受污染节点的最大联通块数量
            print("更新", x)
            max_nodes = size
            ans_index = x
    # 如果ans_index = -1则说明删除任意一个节点都不能降低受感染节点总数量
    return min(initial) if ans_index == -1 else ans_index

graph = [[1,1,0], [1,1,0], [0,0,1]]
initial = [0,1,2]
process(graph, initial)


## 问题19：[从双倍数组中还原原数组](https://leetcode.cn/problems/find-original-array-from-doubled-array/description/)
### 题目
一个整数数组original可以转变成一个双倍数组changed，转变方式为将original中每个元素值乘以2加入数组中，然后将所有元素随机打乱。现在该你一个双倍后的数组changed，请你返回原始数组original。如果无法找到原始数组则返回[]


### 分析
- 将元素进行排序后, 遍历每个元素。我们定义一个字典need_nums, 用于记录后续需要出现在的值。当遍历到元素num时, 如果其在need_nums中存在则元素次数-1(说明前面已经存在num/2), 如果不在，则将2*num放入need_nums中(或者数量+1)，并将num放入到ans中(表示这个num值为元数数组的值)。
- 遍历完成后, 如果need_nums中存在元素则返回空list(说明有num的两倍不存在于changed中, 此changed不是双倍数组)，否则返回ans

In [ ]:
from collections import defaultdict
def process(changed):
    changed_sort = sorted(changed)
    need_nums = defaultdict(int)
    ans = []
    print(changed_sort)
    for i in changed_sort:
        if i in need_nums:
            need_nums[i] -= 1
            if need_nums[i] == 0:
                del need_nums[i]
            print(need_nums)
        else:
            need_nums[i*2] += 1
            ans.append(i)

    return ans if len(need_nums) == 0 else []

changed = [2,1,2,4,2,4]
process(changed)

## 问题20：[将矩阵按对角线排序](https://leetcode.cn/problems/sort-the-matrix-diagonally/description/)
### 题目
阵对角线 是一条从矩阵最上面行或者最左侧列中的某个元素开始的对角线，沿右下方向一直到矩阵末尾的元素。例如，矩阵mat有6行3列，从mat[2][0] 开始的 矩阵对角线 将会经过 mat[2][0]、mat[3][1] 和 mat[4][2] 。
给你一个 m * n 的整数矩阵 mat ，请你将同一条 矩阵对角线 上的元素按升序排序后，返回排好序的矩阵。

### 分析
- 需要构建一种数据结构, 可以在遍历第一边时就将同一对角线的元素储存起来并排序。第二遍时即可挨个pop(0)即可
- 我们可以直到当x-y相等时即为同一对角线元素, 因此我们可以以x-y为key, 储存同一对角线元素的list为value，遍历时使用bisect.insort(sorted_list, new_element)即可在遍历时完成排序。

In [ ]:
from collections import defaultdict

def process(mat):
    m = len(mat)
    n = len(mat[0])
    diag_map = defaultdict(list)
    # 第一次遍历将对角线的元素排序
    for i in range(m):
        for j in range(n):
            bisect.insort(diag_map[i-j], mat[i][j])

    for i in range(m):
        for j in range(n):
            mat[i][j] = diag_map[i-j].pop(0)
    return mat
    


## 问题21：[雇佣 K 位工人的总代价](https://leetcode.cn/problems/total-cost-to-hire-k-workers/description/)
### 题目
给你一个下标从0开始的整数数组costs，其中costs[i]是雇佣第i位工人的代价。同时给你两个整数k和candidates。
- 总共进行k轮雇佣，且每一轮恰好雇佣一位工人。
- 在每一轮雇佣中，从最前面candidates和最后面candidates人中选出代价最小的一位工人(即候选人范围为[:candidates] 和 [-candidates:], 这2*candidates的人里面选)。选出一个工人后, 在costs中排除这个工人(注意每次选择后, 工人下标会变)
- 如果剩余员工数目不足 candidates 人，那么下一轮雇佣他们中代价最小的一人。
- 如果有多位代价相同且最小的工人，选择下标更小的一位工人，一位工人只能被选择一次。

### 分析
- 如果选满足candidates * 2 + k > len(costs), 说明所有的员工都会被加入候选(先后都会), 因此直接返回前k项和即可。
- 使用两个指针left和right, 分别表示已经加入到堆中的员工下标。从0开始遍历, 将(员工成本, 下标)加入到堆中, 左边遍历到candidates-1。右边从len(costs)-1开始遍历, 遍历到len(costs)-candidates下标。完成之后left=candidates-1, right=len(costs)-candidates(注意此时left和right下标指向的元素都已经放入到了堆中了)
- 循环k轮, 每次从堆中弹出最小值, 并拿到下标index, 此时判断index时处于0~left还是属于right~len(costs)-1.
    - 如果处于0~left, 则将(costs[left+1],left+1)加入到堆中, 并更新left += 1
    - 如果处于right~len(costs)-1, 则将(costs[right-1],right-1)加入到堆中, 并更新right -= 1
- 由于排除了第一种情况(被直接返回), 因此之后的操作都不用担心left>=right


In [ ]:
import heapq

def process(costs: List[int], k: int, candidates: int):
    left = -1
    right = len(costs)
    heapq_queue = []  # 创建一个堆
    ans = 0
    # 如果选满足该条件说明所有的员工都会被加入候选
    if candidates * 2 + k > len(costs):
        # 也可以 sum(nsmallest(k, costs))，但效率不如直接排序
        costs.sort()
        return sum(costs[:k])

    # 完成left 和 right 的初始化, 以及堆的初始化
    for _ in range(candidates):
        left += 1
        heapq.heappush(heapq_queue, (costs[left], left))
    for _ in range(candidates):
        right -= 1
        heapq.heappush(heapq_queue, (costs[right], right))
    # 完成后left和right指向的员工都已经放入到了堆中了
    for _ in range(k):
        cost, index = heapq.heappop(heapq_queue)
        ans += cost
        if 0<=index<=left:   # 如果当前员工在左边
            left += 1
            heapq.heappush(heapq_queue, (costs[left], left))
        else:                
            right -= 1
            heapq.heappush(heapq_queue, (costs[right], right))
                
    return ans


## 问题22：[摘樱桃](https://leetcode.cn/problems/cherry-pickup/description/)
### 题目
给你一个n x n的网格 grid ，代表一块樱桃地，每个格子由以下三种数字的一种来表示:
- 0 表示这个格子是空的，所以你可以穿过它。
- 1 表示这个格子里装着一个樱桃，你可以摘到樱桃然后穿过它。
- -1 表示这个格子里有荆棘，挡着你的路。

你根据以下规则进行移动:
- 从位置(0, 0)出发，最后到达(n-1, n-1)，只能向下或向右走，并且只能穿越有效的格(即只可以穿过值为 0 或者 1 的格子)；
- 当到达 (n - 1, n - 1) 后，你要继续走，直到返回到 (0, 0) ，只能向上或向左走，并且只能穿越有效的格子
- 如果在 (0, 0) 和 (n - 1, n - 1) 之间不存在一条可经过的路径，则无法摘到任何一个樱桃.
- 当你经过一个格子且这个格子包含一个樱桃时，你将摘到樱桃并且这个格子会变成空的(值变为0);

请求出能摘到最多樱桃的方案数。

### 分析
- 使用动态规划, 但由于需要返回, 我们可能会快速想到使用两次的DP(动态规划), 先使用一次DP得到最佳路径, 再执行一次DP得到返回的路径, 但在某些特例下这种方式是错误的。(参看题解)
- 因此我们需要**同时**考虑两条路,两个人A和B,都从(0,0)——>(n−1,n−1). 具体的推导分析参看官方题解。
- 定义dp[t][j][k]: 两人从 (0,0) 出发，都走了t步，分别走到(t−j,j)和(t−k,k)位置可以得到的最大樱桃数
- dp[t][j][k] = max(dp[t−1][j][k], dp[t−1][j][k−1], dp[t−1][j−1][k], dp[t−1][j−1][k−1]) + val, 其中val=grid[t−j][j] + grid[t−k][k], 如果k=j, 那么val=grid[t−j][j](其中一个即可, 因为樱桃只能摘一个)
- 为了解决j-1和k-1越界, 我们可以将dp最左边和最上边加一行-inf，即dp[t][j+1][k+1] 表示两人从 (0,0)出发，都走了t步，分别走到 (t−j,j) 和(t−k,k)
- 初始值: dp[0][1][1] = grid[0][0]
- 遍历边界需要认真思考。

In [ ]:
def process(grid):
    n = len(grid)
    # 这里t的范围是[0, 2*n-2], 因为从(0,0)——>(n−1,n−1)只需要走2*n-2步
    dp = [[[float("-inf")] * (n + 1) for _ in range(n + 1)] for _ in range(n * 2 - 1)]
    dp[0][1][1] = grid[0][0]
    for t in range(1, n * 2 - 1):
        for j in range(max(t - n + 1, 0), min(t+1, n)):
                if grid[t-j][j] < 0: continue
                for k in range(j, min(t + 1, n)):  # k从j开始, 到t结束
                    if grid[t-k][k] < 0: continue
                    dp[t][j + 1][k + 1] = max(dp[t - 1][j + 1][k + 1], dp[t - 1][j + 1][k], dp[t - 1][j][k + 1], dp[t - 1][j][k]) + \
                                         grid[t - j][j] + (grid[t - k][k] if k != j else 0)

    return max(dp[-1][n][n], 0)

grid = [[0,1,-1],[1,0,-1],[1,1,1]]
process(grid)

## 问题23：[给植物浇水 II](https://leetcode.cn/problems/watering-plants-ii/description/)
### 题目
Alice 和 Bob 打算给花园里的n株植物浇水。植物排成一行，从左到右进行标记，编号从0到n-1。给你一个下标从0开始的整数数组plants, plants[i]为第i株植物需要的水量。另有两个整数 capacityA 和 capacityB 分别表示 Alice 和 Bob 水罐的容量。
- Alice 按 从左到右 的顺序给植物浇水，从植物 0 开始。Bob 按 从右到左 的顺序给植物浇水，从植物 n - 1 开始。他们 同时 给植物浇水。
无论需要多少水，为每株植物浇水所需的时间都是相同的。
- 如果 Alice/Bob 水罐中的水足以 完全 灌溉植物，他们 必须 给植物浇水。否则，他们重新装满罐子(原地罐装, 无等待)，然后给植物浇水。
- 如果 Alice 和 Bob 到达同一株植物，那么当前水罐中水 更多 的人会给这株植物浇水。如果他俩水量相同，那么 Alice 会给这株植物浇水。

### 分析
- 采用首尾指针即可解决

In [ ]:
def process(plants: List[int], capacityA: int, capacityB: int):
    n = len(plants)
    ans = 0
    # 初始化左右端点
    left = 0 
    right = n-1
    left_wight = capacityA
    right_wight = capacityB
    while right > left:
        if left_wight < plants[left]:  # 如果左边水量不够则增加一次打水
            ans += 1
            left_wight = capacityA
        if right_wight < plants[right]:  # 如果右边水量不够则增加一次打水
            ans += 1
            right_wight = capacityB
            
        left_wight -= plants[left]
        right_wight -= plants[right]
        left += 1
        right -= 1
        
    if left == right:  # 当相遇时, 判断最大值的水量情况
        if max(left_wight, right_wight) < plants[right]:
            ans += 1
    return ans

## 问题24：[收集垃圾的最少总时间](https://leetcode.cn/problems/minimum-amount-of-time-to-collect-garbage/description/)
### 题目
一个字符串数组garbage, 其中garbage[i]="GPM"表示第i个房子的垃圾集合(有一个G,一个P和一个M垃圾), 每个元素可能包含多个相同字符, 每个字符分别表示一单位的金属、纸和玻璃。垃圾车收拾每收拾1单位的任何一种垃圾都需要花费1分钟。给你一个整数数组travel，其中travel[i]表示是垃圾车从房子i行驶到房子i+1需要的分钟数。

目前三辆垃圾车, 每辆垃圾车只能固定收拾一种垃圾。每辆垃圾车都从房子0出发, 按顺序到达每一栋房子, 如果后续房子中不再有对应需要收的垃圾，那么这辆车可以不用继续往后走。请你返回收拾完所有垃圾需要花费的 最少 总分钟数。

注意：任何时刻只有一辆垃圾车处在使用状态。当一辆垃圾车在行驶或者收拾垃圾的时候，另外两辆车不能做任何事情。

### 分析
- 由于不确定每辆车是否需要往后走, 因此我们可以从后方遍历, 保证每辆车在不走多余路。
- 使用标志位表示该辆车是否需要+travel[i-1], 当garbage[i]包含自己需要的垃圾时, 此时标志位需要变为true(表示从此之后的所有travel距离我都需要+时间了)


In [ ]:
def process(garbage, travel):
    flag_G, flag_P, flag_M = False, False, False
    G_cost, P_cost, M_cost = 0, 0, 0

    for index in range(len(garbage)-1, 0, -1):
        garbage_bag = garbage[index]
        G_item, P_item, M_item = garbage_bag.count('G'), garbage_bag.count('P'), garbage_bag.count('M')
        if G_item > 0:
            flag_G = True
        if P_item > 0:
            flag_P = True
        if M_item > 0:
            flag_M = True
        
        if flag_G:
            G_cost += G_item
            G_cost += travel[index-1]
        if flag_P:
            P_cost += P_item
            P_cost += travel[index-1]
        if flag_M:
            M_cost += M_item
            M_cost += travel[index-1]
    
    garbage_bag = garbage[0]
    G_item, P_item, M_item = garbage_bag.count('G'), garbage_bag.count('P'), garbage_bag.count('M')
    G_cost += G_item
    P_cost += P_item
    M_cost += M_item

    return G_cost+P_cost+M_cost

garbage = ["G","P","GP","GG"]
travel = [2,4,3]
process(garbage, travel)

## 问题25：[找出缺失的观测数据](https://leetcode.cn/problems/find-missing-observations/description/)
### 题目
给你一个长度为m的整数数组rolls ，其中 rolls[i] 是第i次观测的值。同时给你两个整数mean和n 。返回一个长度为 n 的数组，包含所有缺失的观测数据，且满足这 n + m 次投掷的 平均值 是 mean 。如果存在多组符合要求的答案，只需要返回其中任意一组即可。如果不存在答案，返回一个空数组。如果不存在这样的观测值则返回[]


### 分析
- 我们可以计算出还需要多少的n_sum, 意味着剩下的n个数的和为n_sum, 我们可以计算出n_sum//n的值, 这个值就是n个数的基准值。
- 若n * 基准值 <= n_sum, 意味着n个数中还有一些数需要加1(并且我们可以知道+1的数一定小于n个, 因为基准值是我们mod得到的)
- python中divmod(被除数, 模)方便的得到模的结果和剩余值。这里的剩余值其实就是表示有多少数值需要+1的
- 为了保证存在观测值, n_sum应该满足>= n 和 <= n*6


In [ ]:
def process(rolls: List[int], mean: int, n: int):
    n_sum = mean * (len(rolls)+n) - sum(rolls)
    if n_sum < n or n_sum > n*6:   # 不满足条件
        return []
    
    base_value, extra_value = divmod(n_sum, n)
    return [base_value]*(n-extra_value) + [base_value + 1] * extra_value

## 问题26：[找出输掉零场或一场比赛的玩家](https://leetcode.cn/problems/find-players-with-zero-or-one-losses/description/)
### 题目
给你一个整数数组matches其中matches[i] = [winner, loser] 表示在一场比赛中winner击败了loser。请返回没有输掉一场比赛的玩家列表, 以及输掉一场比赛的玩家列表,

注意: 每个列表按照升序排列, 并且只考虑那些至少参加过一场比赛的人,

### 分析
- 使用两个set来记录全赢以及输一次的选手

In [ ]:
def process(matches):
    member_set = set()  # 用于记录是否参加了比赛
    winner_set = set()
    loss_one_set = set()
    for i,j in matches:
        if i not in member_set:
            winner_set.add(i)
        # 此处将j放入winner中, 后续会将j移动到loss_one_set中
        if j not in member_set:
            winner_set.add(j)
    
        if j in winner_set:
            winner_set.remove(j)
            loss_one_set.add(j)
        elif j in loss_one_set:
            loss_one_set.remove(j)
        
        member_set.add(i)
        member_set.add(j)

    return [sorted(list(winner_set)), sorted(list(loss_one_set))]
    
matches = [[2,3],[1,3],[5,4],[6,4]]
process(matches)

## 问题27：[找出出现至少三次的最长特殊子字符串 I](https://leetcode.cn/problems/find-longest-special-substring-that-occurs-thrice-i/description/)
### 题目
如果一个字符串仅由单一字符组成，那么它被称为特殊字符串(例如ddd, zz, f)。返回在s中出现至少三次的最长特殊子字符串的长度(出现次数要大于等于3次, 长度要是这些中最长的)


### 分析
- 指针移动并计数, 每次得到最长的相同特殊字串, 比如"ssss"则需要将"s"出现次数+4, "ss"出现次数+3, "sss"出现次数+2, "ssss"出现次数+1, 然后更新最长特殊子字符串的长度.


In [ ]:
from collections import defaultdict

def process(s):
    ans = -1
    count_dict = defaultdict(int)
    left = 0
    right = left + 1
    while right <= len(s):
        # 这里使用right<=len(s)是为了让最后一次循环时left = len(s)-1也可以被计算到
        if right < len(s) and s[right] == s[right-1]:  # right 可以继续前进
            right += 1
            continue

        # 否则更新记录表, right-left为字符s[left]的重复长度
        for count in range(1, right-left+1):
            count_dict[s[left] * count] += right - left + 1 - count
            if count_dict[s[left] * count] >= 3 and ans <= count:
                ans = count
        
        left = right
        right = left + 1

    return ans
        
s = "abcaba"
process(s)


# 问题28：[数组的最大美丽值](https://leetcode.cn/problems/maximum-beauty-of-an-array-after-applying-operation/description/)
### 题目
给你一个下标从0开始的整数数组nums和一个非负整数k。在一步操作中，你可以执行下述指令：
- 在范围有效下标中选择一个 此前没有选过 的下标 i 。
- 将nums[i]替换为范围 [nums[i]- k, nums[i] + k] （指的是下标范围）内的任一整数。
对数组 nums 执行上述操作任意次后，返回数组可能取得的 最大 美丽值(数组中由相等元素组成的最长子序列的长度, 也就是数组中最多相同个数)。

### 分析
- 我们把每个元素都看成一个区间, 只要两个区间有交集, 那么这两个元素一定可以通过指令使得其相等。扩展来看三个区间也是, 只要三个区间中有一个区间和其他两个区间都有交集那么这三个区间的元素都一定可以通过指令使得其三个都相等。更多区间也类似
- 根据上述分析, 我们可以知道这个问题可以转化为, 选出若干个区间，使得这些区间交集不为空。因此原问题可以改为:
    - 排序后，找最长的连续**子数组**，其最大值减最小值不超过 2k。
- 子数组问题可以使用滑动窗口
    - 当c - nums[left] > 2*k 则收缩窗口  

In [ ]:
def process(nums, k):
    nums.sort()
    left = 0
    right = 0 
    ans = 0
    while right < len(nums):
        c = nums[right]
        right += 1
        while c - nums[left] > 2*k:
            left += 1

        # [left, right) 符合要求
        ans = max(ans, right - left)
            
    return ans

nums = [75,15,9]
k = 28
process(nums, k)

        


## 问题29：[最长特殊序列 II](https://leetcode.cn/problems/longest-uncommon-subsequence-ii/description/)
### 题目
给定字符串列表 strs ，返回其中最长的特殊序列的长度。如果最长特殊序列不存在，返回 -1 。特殊序列定义如下: 该序列为某字符串独有的子序列（即不能是其他字符串的子序列）。


### 分析
- 枚举字符串s=strs[i]，判断s是否为其它字符串的子序列，如果不是，则用s的长度更新答案的最大值。
- 可以对strs按长度进行降序, 枚举时如果发现s不为其他字符串的子序列, 则直接返回结果即可

In [ ]:
# 判断s是否为t的子序列
def is_sub_seq(s, t):
    if s == "":
        return True
    index = 0
    for i in t:
        if s[index] == i:  # 如果t的字符等于了s的当前字符, 则可以比较s的下一个字符了
            index += 1
            if index == len(s):
                return True
    return False


def process(strs: list):
    # 按长度降序
    strs.sort(key = lambda x: -len(x))
    for i, s in enumerate(strs):
        # 需要s不是其余所有的其他字符串的子序列, 注意同一位置的字符串不要判断
        if all(not is_sub_seq(s, t) or j==i for j, t in enumerate(strs)):
            return len(s)
    return -1

strs = ["aba","cdc","eae"]
process(strs)

## 问题30：[价格减免](https://leetcode.cn/problems/apply-discount-to-prices/description/)
### 题目
将一个句子中所有"$100"、"$23" 和 "$6" 表示价格的单词, 将其价格减免discount%。并更新该单词到句子中。所有更新后的价格应该表示为一个恰好保留小数点后两位的数字。返回表示修改后句子的字符串。

注意: 例如 "$100"、"$23" 和 "\$6" 表示价格，而 "100"、"\$" 和 "$1e5 不是。

### 分析
- 将字符串以" "分割后判断每一个单词, 如果当单词为$开头且后续字符串均为数字则说明此为一个价格
- str.isdigit()方法用于检查字符串中的所有字符是否都是数字（0-9）

In [ ]:
def process(sentence, discount):
    ans = []
    words = sentence.split(" ")
    d = 1 - discount/100
    for word in words:
        if word[0] == "$" and word[1:].isdigit():
            cost = int(word[1:])
            ans.append(f"${cost * d :.2f}")
        else:
            ans.append(word)
    return " ".join(ans)

sentence = "ka3caz4837h6ada4 r1 $602"
discount = 9
process(sentence, discount)

## 问题31：[找到矩阵中的好子集](https://leetcode.cn/problems/find-a-good-subset-of-the-matrix/description/)
### 题目
给你一个下标从 0 开始大小为 m x n 的二进制矩阵 grid(只包含0,1两种数值，且最多有5列)。从原矩阵中选出若干行构成一个**行**的非空子集(行子集: 是删除grid中某些行后，剩余行构成的元素集合), 如果子集中 任何一列的元素和 <= floor(行数 / 2)，那么我们称这个子集是好子集。

请你返回一个整数数组，它包含好子集的**行下标**, 请你将子集中的元素 升序 返回。如果有多个好子集，你可以返回任意一个。如果没有好子集，请你返回一个空数组。

### 分析
- [参考解析](https://leetcode.cn/problems/find-a-good-subset-of-the-matrix/solutions/2305490/xiang-xi-fen-xi-wei-shi-yao-zhi-duo-kao-mbl6a/?envType=daily-question&envId=2024-06-25)
- 经过参考解析我们可以知道在列数小于5时, 好子集的行数最多为2行。而针对2行的情况, 我们只需要保证2行中每一列的AND运算后为false即可。
- 由于需要计算列元素的AND运算, 因此我们可以使用一个hash表(键为行元素二进制化后的结果01011, 值为行号), 如果有某一行二进制后全为0, 则直接返回该行即可。否则将其存入hash表中
- 遍历hash表中任何两个元素的键做AND操作(也就是对列元素做AND了), 如果存在为0的情况, 则返回这两个键值对的行号(注意需要排序); 否则返回[]


In [ ]:
def process(grid):
    mask_hash = {}
    for index, row in enumerate(grid):
        mask = 0
        for column_index, value in enumerate(row):
            # 通过位运算将每一行的值二进制化
            mask |= value << column_index
        
        if mask == 0:  # 如果该行的所有值为0说明找到了好子集直接返回
            return [index]
        # 将 行二进制-行号 储存进哈希表
        mask_hash[mask] = index
    # 遍历哈希表中任意两个元素
    for one_value, one_index in mask_hash.items():
        for two_value, two_index in mask_hash.items():
            if one_value & two_value == 0:
                return sorted((one_index, two_index))
    return []

grid = [[0,1,1,0],[0,0,0,1],[1,1,1,1]]
process(grid)

## 问题32：[特别的排列](https://leetcode.cn/problems/special-permutations/description/)
### 题目
给你一个下标从 0 开始的整数数组 nums ，它包含 n 个 互不相同 的正整数。如果 nums 的一个排列满足以下条件，我们称它是一个特别的排列:
对于排列中的任意相邻两个数,都满足要么nums[i] % nums[i+1] == 0 ，要么 nums[i+1] % nums[i] == 0 。请你返回特别排列的总数目，由于答案可能很大，请将它对 10**9 + 7取余后返回。

注意: nums长度小于等于14

### 分析
- 我们可以遍历nums中任意两个元素, 判断他们是否满足 x % y == 0 或者 y % x == 0, 如果满足则说明x, y之间存在一条边。遍历完成后即可得到邻接矩阵。(可以使用itertools.combinations(list, 2)的方法来得到组合数)
- 得到邻接矩阵后我们可以使用dfs(深度优先遍历) + 回溯法 得到最后的结果, 使用visit_set集合来记录已经访问过的位置。

### 优化
- 考虑这种情况：
    - 目前生成的排列是 p=[2,4,1,_,_]，现在递归到倒数第二个位置，那么visit_set={1,2,4}。
    - 目前生成的排列是 p=[4,2,1,_,_]，现在递归到倒数第二个位置，那么visit_set={1,2,4}。
    - 上述两种情况接下来得到的是同样的子问题, 因此我们可以使用@cache来优化递归
- 为了能使用@cache, 我们需要使用位运算来代替visit_set集合(因为dfs中有集合是无法@cache)。我们使用"1111"二进制表示可选下标集合
    - u = (1 << 3) - 1 : "111" 表示可选下标集合, 这里标识下标0,1,2都可选
    - index被选取: "11111" ^ (1 << index), "111" 与 "100"进行异或运算得到 "011", 这样得到的表示可选下标只有0,1了。因此我们可以使用"11111" ^ (1 << index)的方式得到还有那些是可选的下标
    - 判断index能否被选: "101" >> index & 1 如果该值=1则表示index可以被选取, 如果该值为0则表示index不可被选取



In [ ]:
import itertools
from collections import defaultdict
from functools import cache


def process(nums):
    @cache
    def dfs(root, select_state):
        nonlocal graph
        if select_state == 0: # 状态为0则表示, 没有待选取的位置了
            return 1
        # 得到可选取的邻居下标
        neighbors = graph[root]
        count = 0
        for neighbor in neighbors:
            if select_state >> neighbor & 1 == 0: # 表示neighbor这个位置不可选取
                continue
            count += dfs(neighbor, select_state ^ (1 << neighbor))  # 表示选取neighbor这个位置
        return count

    # 这里的图是的键和值是指nums中的下标
    graph = defaultdict(list)
    for x, y in itertools.combinations(enumerate(nums), 2):
        if x[1] % y[1] == 0 or y[1] % x[1] == 0:
            graph[x[0]].append(y[0])
            graph[y[0]].append(x[0])

    # 如果邻接矩阵为为空则直接返回0
    if len(graph) == 0:
        return 0
    select_state = (1 << len(nums)) - 1
    ans = 0
    print(graph)
    for index in range(len(nums)):
        ans += dfs(index, select_state ^ (1 << index))
    return ans % (10**9 + 7)

nums = [20,100,50,5,10,70,7]
process(nums)

## 问题33：[优美的排列](https://leetcode.cn/problems/beautiful-arrangement/description/)
### 题目
假设有从 1 到 n 的 n 个整数。用这些整数构造一个数组 perm(下标从 1 开始)，只要满足下述条件之一，该数组就是一个优美的排列 ：
- perm[i] 能够被 i 整除
- i 能够被 perm[i] 整除

给你一个整数 n ，返回可以构造的优美排列的数量 。


### 分析
- 与[特别的排列](https://leetcode.cn/problems/special-permutations/description/)一样, 我们需要使用记忆化搜索。更简单的是我们并不需要提前处理得到"邻接矩阵"
- 本题尝试使用二进制表示"已使用位置集合"
    - 初始状态: "00000" 表示所有位置都未被使用
    - 判断index能否使用: select_state >> index & 1 等于0表示index还未被访问, 等于1表示index已被访问
    - index使用: select_state | (1 << index) 表示index位置被访问后
- 注意本题是从下标1开始, 所以num-1才表示select_state中的位置

In [ ]:
from functools import cache

def process(n):
    u = (1 << n) - 1  # 长度为n

    @cache
    def dfs(select_state):
        if select_state == u:  # 表示所有的数字都被使用了
            return 1
        res = 0
        # 选取当前数字的下标数值(表示当前需要选取数字的位置为i)
        i = select_state.bit_count() + 1

        for num in range(1, n+1):
            if select_state >> (num-1) & 1 == 1: # 说明这个数字num已经被访问过了
                continue
            if num % i == 0 or i % num == 0:
                res += dfs(select_state | (1 << (num-1)))  # select_state | (1 << (num-1))表示数字num被使用了
        return res
    
    return dfs(0)  # 初始状态为0

## 问题34：[执行子串操作后的字典序最小字符串](https://leetcode.cn/problems/lexicographically-smallest-string-after-substring-operation/description/)
### 题目
给你一个仅由小写英文字母组成的字符串s。你可以完成以下行为：
选择s的任一非空子字符串(包括整个字符串), 将选择的字符串中的每一个字符替换为英文字母表中的前一个字符。例如,'b' 用 'a' 替换，'a' 用 'z' 替换。返回执行上述操作 恰好一次 后可以获得的字典序最小的字符串。

### 分析
- 使用贪心即可, 遍历字符串，找到第一个不为a的字符开始变换(触发逻辑), 一直变换到下一个为a的字符为止
- chr(ord("字母") - 1) 可以实现变化为前一字母
- 注意: 如果s全为a，由于题目要求必须操作一次，可以把最后一个 a 改成 z。

In [ ]:
def process(s):
    s_list = list(s)

    for index, word in enumerate(s):
        if word == "a":
            continue
        
        # 直到遍历到第一个非a为止
        for j in range(index, len(s)):  # 第一个非a的坐标开始向后遍历
            if s[j] == "a":  # 遇到第二个a则跳出循环
                break
            else:
                s_list[j] = chr(ord(s[j]) - 1)
    
        return "".join(s_list)
    # 如果能走到这里, 说明字符串全是a
    s_list[-1] = "z"
    return "".join(s_list)


s = "aa"
process(s)

## 问题35：[目标和](https://leetcode.cn/problems/target-sum/description/)
### 题目
给你一个非负整数数组 nums 和一个整数 target。向数组中的每个整数前添加 '+' 或 '-' ，然后串联起所有整数，可以构造一个 表达式: 例如，nums = [2, 1]，可以在 2 之前添加 '+' ，在 1 之前添加 '-' ，然后串联起来得到表达式 "+2-1" 。
返回可以通过上述方法构造的、运算结果等于 target 的不同 表达式 的数目。

### 分析
- 每个元素前+或者-, 可以使用递归 + 回溯方式遍历完所有的情况。dfs(index, target), 表示从index开始, 获得target有多少种组合

### 优化
- 转化为0-1背包模型【使用DP(动态规划)解决】, 我们可能会很自然的想到使用dp[i][target]的定义方式(截止到i位置, 目标和为target的情况数), 但注意到本题中target是可能为负数的因此不能作为背包问题的"容量". 我们进行一些转换, 假设nums的元素和为s，添加正号的元素之和为p, 那么p, 其实我们 p = (s+target)/2, 即问题变成了: 从nums中选一些数, 和为p的情况种类数。
    - 由于p一定为整数, 因此如果s+target为负数或者为奇数, 那么一定不存在这样的方案, 因此直接返回0
    - dp[i][c]: 截止到第i个数字(i从1开始), 目标和为c的情况数
- 由于是"恰好"等于c的方案数, 选与不选两者是互斥，因此状态转移方程为: dp[i][c] = dp[i-1][c] + dp[i-1][c-num]
- 边界条件为: dp[i][0] = 1 (由于i从1开始, 这里的0为表示目标值为0, 不选则为0)

In [ ]:
from functools import cache

nums = [1,1,1,1,1]
target = 3

@cache
def dfs(index, target):
    if index >= len(nums):
        if target == 0:
            return 1
        else:
            return 0 
    
    num = nums[index]
    ans = 0
    ans += dfs(index+1, target-num)  # 表示当前+num, 则剩余的目标为target-num
    ans += dfs(index+1, target+num)  # 表示当前-num, 则剩余的目标为target-num
    return ans

dfs(0, target)


In [ ]:
def process(nums, target):
    target += sum(nums)
    if target < 0 or target % 2 != 0 :  # 如果小于0或者不为偶数则直接返回方案0
        return 0 
    target = target // 2
    n = len(nums)

    dp = [[0] * (target+1) for _ in range(n+1)]
    # 初始化
    for i in range(n+1):
        dp[i][0] = 1

    for i in range(1, n+1):
        num = nums[i-1] # 当前数字, 注意i从1开始
        for j in range(target+1):  # 这必须要从0开始
            if j < num:  # 不能选这个num
                dp[i][j] = dp[i-1][j]
            else:
                dp[i][j] = dp[i-1][j] + dp[i-1][j-num]
                
    return dp[n][target]
        
nums = [1,1,1,1,1]
target = 3
process(nums, target)

In [ ]:
## 空间优化版
def process(nums, target):
    target += sum(nums)
    if target < 0 or target % 2 != 0 :  # 如果小于0或者不为偶数则直接返回方案0
        return 0 
    target = target // 2
    n = len(nums)

    dp = [0] * (target+1)
    # 初始化
    dp[0] = 1

    for i in range(1, n+1):
        num = nums[i-1] # 当前数字, 注意i从1开始
        for j in range(target, num-1, -1):  # 从后往前滚动, 保证判断j时使用的都是前一轮的j-num
            dp[j] = dp[j] + dp[j-num]
                       
    return dp[target]

nums = [1,1,1,1,1]
target = 3
process(nums, target)

## 问题36：[最大化一张图中的路径价值](https://leetcode.cn/problems/maximum-path-quality-of-a-graph/description/)
### 题目
给你一张无向图(可能存在环)，图中有n个节点，节点编号从0到n-1。values[i]表示第i个节点的价值。 edges[j]=[u, v, time]表示节点u和v之间有一条需要 time秒才能通过的无向边。最后，给你一个整数maxTime。请你返回一条合法路径的 最大 价值。

合法路径: 从节点0开始，最终回到节点0，且花费的总时间不超过maxTime秒的一条路径。注意每个节点可多次访问, 但价值只能计算一次

### 分析
- 针对这种value和edges的我们遍历边得到邻接图, 不过邻接图的元素为(相邻节点, 价值)
- 使用深度遍历dfs, 当遍历到x时判断是否回到了0, 如果是则更新最优结果, 否则遍历x的所有邻居节点。由于每个节点的价值只能计算一次, 因此我们需要使用visited数组来记录是否该节点的价值已经被计算过了(暴力搜索)

In [ ]:
def process(values, edges, maxTime):
    ans = 0
    n = len(values)
    visited = [False] * n  # visited[i]=false表示该节点没有被遍历过
    graph = [[] for _ in range(n)]
    # 得到邻接图
    for u, v, t in edges:
        graph[u].append((v, t))
        graph[v].append((u, t))

    # 遍历到x时(已计算), 总时间为sum_time, 总价值为sum_value
    def dfs(x, sum_time, sum_value):
        nonlocal ans
        if x == 0:
            ans = max(ans, sum_value) # 更新最优结果
            # 注意此处不能return, 因为你违法判断这个x时第一次还是第二次访问到(我们不需要记录第几次访问, 只用记录回到了0)
        
        for y, t in graph[x]: # 遍历x的邻居节点
            # 因为允许返回, 因此不能判定父节点

            # 如果去向y节点耗时已经大于maxTime了则不能去y节点, 通过这种方式保证不会栈溢出
            if sum_time + t > maxTime:  
                continue

            if visited[y]:  # 说明已经访问过了, 不能重复计算value, 但耗时得增加
                dfs(y, sum_time + t, sum_value)
            else:
                visited[y] = True
                dfs(y, sum_time+t, sum_value+values[y])
                # 注意回溯
                visited[y] = False

    # 注意初始化条件
    visited[0] = True
    dfs(0, 0, values[0])

    return ans

values = [5,10,15,20]
edges = [[0,1,10],[1,2,10],[0,3,10]]
maxTime = 30

process(values, edges, maxTime)

## 问题37：[质数的最大距离](https://leetcode.cn/problems/maximum-prime-difference/description/)
### 题目
给你一个整数数组 nums。返回两个（不一定不同的）质数在 nums 中 下标 的 最大距离。如果只存在一个质数则返回0

注意: 数值范围为[1,100]


### 分析
- 两次遍历, 分别找到首质数和尾部质数
- 由于质数范围有限, 则可以先创建一个质数集

In [ ]:
def isPrime(n):
    """
    n: 必须大于2
    """
    i = 2
    while i*i <= n:
        if n%i == 0:
            return False
        i += 1
    return True

def process(nums):
    # 得到质素集
    prime_set = set()
    for i in range(2, 101):
        if isPrime(i):
            prime_set.add(i)
    
    ans = 0
    for index, num_1 in enumerate(nums):
        if num_1 not in prime_set:
            continue
        # 来到这里说明找到了第一个质数的下标, 从当前下标开始遍历后续的
        for index_2, num_2 in enumerate(nums[index:]):
            if num_2 in prime_set:  # 如果为质数则更新最大值
                ans = max(ans, index_2)
        return ans
    
nums = [4,2,9,5,3]
process(nums)

## 问题38：[哈沙德数](https://leetcode.cn/problems/harshad-number/description/)
### 题目
如果一个整数能够被其各个数位上的数字之和整除，则称之为 哈沙德数（Harshad number）。给你一个整数 x 。如果 x 是 哈沙德数 ，则返回 x 各个数位上的数字之和，否则，返回 -1 。

### 分析
- 求一个数的位数和
- 使用// 和 % 即可

In [ ]:
def process(num):
    pos_sum = 0
    num_2 = num
    while num_2 != 0:
        pos_sum += num_2 % 10
        num_2 //= 10

    return pos_sum if num % pos_sum == 0 else -1


## 问题39：[交替子数组计数](https://leetcode.cn/problems/count-alternating-subarrays/description/)
### 题目
给你一个二进制数组nums(只包含0,1两个数字)，如果一个子数组中不存在两个相邻元素的值相同的情况，我们称这样的子数组为交替子数组。请返回数组 nums中交替子数组的数量。

### 分析
- 如果一个交替数组的长度为n, 那么其交替子数组的个数为 1+...+n = (1+n)*n // 2. 因此我们只需要将二进制数组遍历时记录最长交替数组即可, 当下一个字母不能与前一个字母组成交替子数组则更新答案

In [ ]:
def process(nums):
    ans = 0
    n = 1
    for i in range(len(nums)-1):
        if nums[i] == nums[i+1]:
            ans += ((1+n)*n) // 2
            n = 1   # 将nums[i+1]单独为一个子数组
        else:   
            n += 1  # 将nums[i+1]加入到前子数组中
    # 退出循环后还有计算一次
    ans += ((1+n)*n) // 2
    return ans

nums = [1,1]
process(nums)

## 问题40：[寻找数组的中心下标](https://leetcode.cn/problems/find-pivot-index/description/)
### 题目
给你一个整数数组nums(元素有正有负), 请计算数组的中心下标。中心下标: 是数组的一个下标, 其左侧所有元素相加的和等于右侧所有元素相加的和(均不计算中心下标的数字)。如果数组有多个中心下标，应该返回 最靠近左边 的那一个。如果数组不存在中心下标, 返回 -1 。

### 分析
- 分别从左往右和从右往左遍历得到left_sum和right_sum数组, left_sum[i] 表示[:i]的求和结果, right_sum[i]表示[i+1:]的求和结果,然后再遍历left_sum和right_sum, 从左往右第一个相等的元素返回index

In [ ]:
def process(nums):
    n = len(nums)
    left_sum = [0] * n
    right_sum = [0] * n
    for i in range(1, n):
        left_sum[i] = left_sum[i-1] + nums[i-1]
        right_sum[i] = right_sum[i-1] + nums[n-i]
    # 将right_sum进行倒序
    right_sum = right_sum[::-1]
    for index in range(n):
        if left_sum[index] == right_sum[index]:
            return index
    
    return -1
nums = [1, 7, 3, 6, 5, 6]
process(nums)

## 问题41：[统计移除递增子数组的数目 I](https://leetcode.cn/problems/count-the-number-of-incremovable-subarrays-i/description/)
### 题目
给你一个下标从 0 开始的 正 整数数组 nums 。如果 nums 的一个子数组满足：移除这个子数组后剩余元素 严格递增 ，那么我们称这个子数组为 移除递增 子数组。比方说，[5, 3, 4, 6, 7] 中的 [3, 4] 是一个移除递增子数组，因为移除该子数组后，[5, 3, 4, 6, 7] 变为 [5, 6, 7] ，是严格递增的。请你返回 nums 中 移除递增 子数组的总数目。

请用O(n)时间复杂度
### 分析
- 讨论前缀[0, i]和后缀[r, n-1]
    - 假设前缀的[0, i]位置是前缀中最长的递增子数组, 那么我们可以删除的子数组为[i+1, n-1], [i, n-1]...[0, n-1], 一共i+2个
    - 假设我们移除的子数组为中间部分的 [0, i] [i+1, r-1] [r, n-1]即[i+1, r-1]被移除。此时为了能使得剩下的子数组能形成严格递增，则要求前缀[0, i]中最后一个元素nums[i] < nums[r]后缀中的第一个元素。并且还必须要求[0, i]和[r, n-1]均为严格单调递增的。由于我们已经在上一种情况的遍历中已经得到最大的[0, i]的严格递增字串了, 因此我们需要满足的是[r, n-1]均为严格单调递增以及nums[i] < nums[r]
    - 我们可以在逆序遍历中先确保找[r, n-1]是单调递增的(直到找到最大的单调区间[r, n-1])。在每次遍历中，得到一个[r, n-1]单调区间后, 然后将i不断减少, 直到nums[i1] < nums[r], 此时我们知道删除[i1+1, r-1], [i1, r-1], [i1-1, r-1]....[0, r-1]共i1+2个子数组都可以满足要求(并且注意这些子数组与第一种情况是不同的)

In [ ]:
def process(nums):
    ans = 0
    i = 0
    n = len(nums)
    # 先找出最大的[0, i]单调递增区间
    while i < n-1 and nums[i] < nums[i+1]:  
        i += 1
    # 如果i==n-1时说明[0, n-1]均为严格单调递增的,所有子区间都可以被删除
    if i == n-1:
        return (1+n) * n // 2
    else:
        ans = i+2  # 否则加上可删除的全部后缀[i+1, n-1], [i, n-1]...[0, n-1], 一共i+2个
    
    # 第二次遍历找到[r, n-1]严格单调区间
    r = n-1
    while r == n-1 or nums[r+1] > nums[r]: # 当r=n-1时也是单调递增的
        # 找到i1, 使得nums[i1] < nums[r]
        while nums[i] >= nums[r] and i >= 0:
            i -= 1
        
        # 此输满足[0, i1], [r, n-1]严格递增, 且nums[i] < nums[r]
        ans += i+2  # 如果i=-1, 其实删除的是[0, r-1]这一个区间
        print(i+2)
        
        # 判断下一个r
        r -= 1

    return ans


nums = [6,5,7,8]
process(nums)


## 问题42：[最小数字游戏](https://leetcode.cn/problems/minimum-number-game/description/)
### 题目
你有一个下标从0开始、长度为偶数的整数数组nums，同时还有一个空数组arr 。Alice 和 Bob 决定玩一个游戏，游戏中每一轮 Alice 和 Bob 都会各自执行一次操作。游戏规则如下：
- 每一轮，Alice 先从 nums 中移除一个最小元素，然后Bob执行同样的操作。
- 接着，Bob会将移除的元素添加到数组arr中，然后Alice也执行同样的操作。
- 游戏持续进行，直到 nums 变为空。

返回结果数组 arr 。

### 分析
- 先将nums进行排序，然后间隔遍历, 得到ans

In [ ]:
def process(nums):
    nums.sort()
    ans = []
    for i in range(0, len(nums), 2):
        ans.append(nums[i+1])
        ans.append(nums[i])
    return ans

nums = [5,4,2,3]
process(nums)

## 问题43：[判断一个数组是否可以变为有序](https://leetcode.cn/problems/find-if-array-can-be-sorted/description/)
### 题目
给你一个下标从 0 开始且全是 正 整数的数组 nums 。
一次操作中，如果两个相邻元素在二进制下数位为1的数目相同，那么你可以将这两个元素交换。你可以执行这个操作任意次（也可以0次）。

如果你可以使数组变有序，请你返回 true ，否则返回 false 

### 分析
- 我们可以使用bin(n).count('1')(bin将n转化为二进制字符串表示, 在对字符传中计数)
- 由于1的相同,可以交换, 那么意味着当1的数目相同的元素一定可以拍成有序, 那因此我们根据1数目连续相同可以把nums分成多个区域, 我们子啊每个区域中维护当前区域的最大值cur_max, 当遍历到下一个区域时, 我们判断当前区域中的值是否都会大于pre_max(前一个区域的最大值), 注意需要先将pre_max=cur_max, 再判断。

In [ ]:
# 计算n的1的个数
def count_ones_in_binary(n):
    count = 0
    while n:  # n非0则继续遍历
        count += 1
        n &= n-1  # 将n的最右边一个1变为0
    return count

def process(nums):
    pre_max = -1
    cur_max = nums[0]
    for i in range(1, len(nums)):
        if count_ones_in_binary(nums[i-1]) == count_ones_in_binary(nums[i]):  # 说明是一个区域的
            if nums[i] >= pre_max:  # 说明是可以有序的
                cur_max = max(cur_max, nums[i])
            else:
                return False
        else:
            pre_max = cur_max       # 跨区域时, 需要先将上个区域的cur_max保存下来
            if nums[i] >= pre_max:  # 说明是可以有序的
                cur_max = max(cur_max, nums[i])
            else:
                return False

    return True

nums = [20,16]
process(nums)


## 问题44：[保持城市天际线](https://leetcode.cn/problems/max-increase-to-keep-city-skyline/description/)
### 题目
给你一座由 n x n 个街区组成的城市，每个街区都包含一座立方体建筑。给你一个下标从 0 开始的 n x n 整数矩阵 grid ，其中 grid[r][c] 表示坐落于 r 行 c 列的建筑物的 高度 。城市的 天际线 是从远处观察城市时，所有建筑物形成的外部轮廓（每一行,每一列的最大值）。从东、南、西、北四个主要方向观测到的 天际线 可能不同。

在不改变从任何主要方向观测到的城市天际线的前提下，返回建筑物可以增加的最大高度增量总和。(即不能增加高度后不能改变每一行每一列的最大值)
### 分析
- 第一次遍历街区时计算row_max, col_max用来记录每一行列的最大值
- 第二次遍历到(i,j)位置时, ans += min(row_max[i], col_max[j]) - grid[i][j]
- 可以只用迭代公式更快的得到row_max, col_max

In [ ]:
def process(grid):
    n = len(grid)
    row_max = [max(row) for row in grid]
    col_max = [max(col) for col in zip(*grid)]
    ans = 0
    for i in range(n):
        for j in range(n):
            ans += min(row_max[i], col_max[j]) - grid[i][j]
    return ans

grid = [[3,0,8,4],[2,4,5,7],[9,2,6,3],[0,3,1,0]]
process(grid)

## 问题45：[账户合并](https://leetcode.cn/problems/accounts-merge/description/)
### 题目
给定一个列表 accounts，每个元素 accounts[i] 是一个字符串列表，其中第一个元素 accounts[i][0]是名称 (name)，其余元素是 emails 表示该账户的邮箱地址。
- 如果邮箱存在交集, 那么这两个人一定是同一个人
- 可能出现同名, 因此不能使用名称来判断是否为同一个人

合并账户后，按以下格式返回账户: 每个账户的第一个元素是名称，其余元素是按字符ASCII顺序排列的邮箱地址。账户本身可以以任意顺序返回。

### 分析
- 类似于在求解联通分支个数, 集合个数, 典型的并查集类型题. 对于这种题目我们可以使用并查集方式将其合并, 也可以使用图+dfs的方法.
- 并查集模板
```python
class UnionFind:
    def __init__(self, n) -> None:  # 使用并查集需要知道有多少个节点
        self.pre = list(range(n))
        self.size = [1] * n         # size[i]表示节点i下的子节点数

    # 使用路径压缩find
    def find(self, x):
        fx = self.pre[x]
        if fx == x:
            return x
        else:
            if fx != self.pre[fx]:  # 如果fx不是代表元, 则需要将fx的子节点个数-1
                self.size[fx] -= 1
            self.pre[x] = self.find(fx)
            return self.pre[x]
    
    # 将节点x, 与节点y合并(返回是否完成合并, 如果为False表明未合并)
    def union(self, x, y) -> bool:
        fx = self.find(x)
        fy = self.find(y)
        if fx == fy:    # 说明为x, y在同一个代表元下
            return False
        elif self.size(fx) > self.size(fy):    # 合并后将少节点的集合合并到多节点中
            self.pre[fy] = fx
            self.size[fx] += self.size[fy]
        elif self.size(fx) < self.size(fy):
            self.pre[fx] = fy
            self.size[fy] += self.size[fx]
        return True
```
- 我们使用并查集需要明白什么是代表元, 本题中我们需要合并的是用户, 因此我们的节点可以是accounts中的用户下标, 将这些下标进行合并后最后得到的就是多少个集合群

In [ ]:
from collections import defaultdict

class UnionFind:
    def __init__(self, n) -> None:  # 使用并查集需要知道有多少个节点
        self.pre = list(range(n))
        self.size = [1] * n         # size[i]表示节点i下的子节点数

    # 使用路径压缩find
    def find(self, x):
        fx = self.pre[x]
        if fx == x:
            return x
        else:
            if fx != self.pre[fx]:  # 如果fx不是代表元, 则需要将fx的子节点个数-1
                self.size[fx] -= 1
            self.pre[x] = self.find(fx)
            return self.pre[x]
    
    # 将节点x, 与节点y合并(返回是否完成合并, 如果为False表明未合并)
    def union(self, x, y) -> bool:
        fx = self.find(x)
        fy = self.find(y)
        if fx == fy:    # 说明为x, y在同一个代表元下
            return False
        elif self.size[fx] >= self.size[fy]:    # 合并后将少节点的集合合并到多节点中
            self.pre[fy] = fx
            self.size[fx] += self.size[fy]
        elif self.size[fx] < self.size[fy]:
            self.pre[fx] = fy
            self.size[fy] += self.size[fx]
        return True


def process(accounts):
    un_find = UnionFind(len(accounts))
    d = {}  # email: index 用于记录email对应的用户下标
    for i, (_, *emails) in enumerate(accounts):
        for email in emails:
            if email in d:   # 说明当前用户i, 是un_find中的某个用户
                un_find.union(i, d[email])   # 将用户i和j合并
            else:
                d[email] = i   # 将这个email与用户的映射保存下来
    
    # 完成un_find的合并, 接下来找到每个用户的代表元(根用户), 将所有的email合并
    g = defaultdict(set)
    for j, (_, *emails) in enumerate(accounts):
        root = un_find.find(j)  # 找到根用户
        g[root].update(emails)    # set1 U set2
    print(un_find.pre)
    return [[accounts[index][0]] + sorted(emails) for index, emails in g.items()]


accounts = [["John","johnsmith@mail.com","john_newyork@mail.com"],["John","johnsmith@mail.com","john00@mail.com"],["Mary","mary@mail.com"],["John","johnnybravo@mail.com"]]
process(accounts)


## 问题47：[访问消失节点的最少时间](https://leetcode.cn/problems/minimum-time-to-visit-disappearing-nodes/description/)
### 题目
给你一个二维数组edges表示一个n个点的无向图，其中edges[i]=[ui, vi, lengthi]表示节点ui和节点vi之间有一条需要lengthi单位时间通过的无向边。同时给你一个数组disappear，其中disappear[i]表示节点i从图中消失的时间点，在那一刻及以后，你无法再访问这个节点。

请你返回数组answer ，answer[i]表示从节点0到节点i需要的最少单位时间。如果从节点0出发无法到达节点i ，那么answer[i]为-1。


### 分析
- 在Dijkstra算法更新最新边的时候, 判断是否可以更新如果当前距离大于等于限制距离则不进行更新。

In [ ]:
import heapq

def process(n, edges, disappear):
    graph = [[] for _ in range(n)]  # 稀疏图用邻接表
    for x, y, wt in edges:
        graph[x].append((y, wt))
        graph[y].append((x, wt))
    
    # 接下来是Dijkstra算法
    ans = [-1 for _ in range(n)]
    ans[0] = 0     # 初始点的距离为0
    visitSet = set()     # 记录使用过的点
    pqueue = [(0, 0)]          # 用来存放还没被完全更新的节点

    while pqueue:      # 当pqueue为空时意味着已经被更新完毕了，则不用再循环了
        distance, minNode = heapq.heappop(pqueue)
        if distance > ans[minNode]:  # Node之前出堆过可以跳过
            continue

        edges = graph[minNode]
        for node, weight in edges:
            if node in visitSet: # 访问过的邻居不用再遍历
                continue
            new_distance = weight + distance
            if (new_distance < ans[node] or ans[node] == -1) and new_distance < disappear[node]:
                ans[node] = new_distance
                # 需要把待更新的点压进pqueue
                heapq.heappush(pqueue, (ans[node], node))
        # 将这个节点放入到已使用节点中 
        visitSet.add(minNode)
    return ans

n = 10
edges = [[8,9,9],[2,9,3],[7,2,5],[0,9,3],[0,9,7],[8,5,2],[2,8,1],[9,5,3],[0,5,10],[7,7,10]]
disappear = [5,7,19,14,4,19,10,18,11,14]
process(n, edges, disappear)
    


## 问题48：[得到更多分数的最少关卡数目](https://leetcode.cn/problems/minimum-levels-to-gain-more-points/description/)
### 题目
给你一个长度为 n 的二进制数组 possible。Alice 和 Bob 正在玩一个有 n 个关卡的游戏，游戏中有一些关卡是 困难 模式，其他的关卡是 简单 模式。如果 possible[i] == 0 ，那么第 i 个关卡是 困难 模式。一个玩家通过一个简单模式的关卡可以获得+1分，通过困难模式的关卡将失去-1分。

Alice先手从第0级开按顺序完成一些关卡，然后Bob会完成剩下的所有关卡。请返回Alice获得比Bob更多的分数所需要完成的最少关卡数目。

注意: 每个玩家必须完成1个
### 分析
- 由于A和B会完成所有关卡, 所以先遍历数组得到得分总和, 再第二次遍历数组时更新A的得分A_score和B的得分B_score = total - A_score
- 这也叫"前缀和"类型题

In [ ]:
def process(possible):
    total = sum([-1 if i==0 else 1 for i in possible])
    A_score = 0
    B_score = total - A_score
    ans = 0
    for i in possible:
        A_score += (-1 if i==0 else 1)
        B_score = total - A_score
        ans += 1
        if A_score > B_score and ans < len(possible):
            return ans
    return -1

possible = [1,1]
process(possible)

## 问题49：[将石头分散到网格图的最少移动次数](https://leetcode.cn/problems/minimum-moves-to-spread-stones-over-grid/description/)
### 题目
给你一个大小为 3 * 3 ，下标从 0 开始的二维整数矩阵 grid ，分别表示每一个格子里石头的数目。网格图中总共恰好有 9 个石头，一个格子里可能会有多个石头。每一次操作中，你可以将一个石头从它当前所在格子移动到一个至少有一条公共边的相邻格子。请你返回每个格子恰好有一个石头的 最少移动次数 。

### 分析
- 粗暴的方式: 将超过1的格子坐标记录下来, 我们最终的目的就是把这些多余的石头坐标, 放到0坐标位置, 我们再记录下0坐标的位置。使用全排列将这些位置一一匹配最后计算(曼哈顿距离)出最小值
- 匹配全排列的技巧: 使用itertools.permutations(可迭代对象), 将会返回全排列集集合, 再对另一个匹配组一一匹配即可

In [ ]:
import itertools

def process(grid):
    n = 3
    from_pos = []  # 多余的石头坐标
    to_pos = []    # 为0的坐标
    for i in range(n):
        for j in range(n):
            if grid[i][j] > 1:
                from_pos.extend([(i,j) for _ in range(grid[i][j]-1)])   # 将多余的石头放到列表中
            elif grid[i][j] == 0:
                to_pos.append((i,j))
    ans = float("inf")
    for from_pailie in itertools.permutations(from_pos):
        # 每个from_pailie = [(),(),...] 一种排列
        tmp = 0
        for (x1, y1), (x2, y2) in zip(from_pailie, to_pos):  # 计算这种方式得到的结果
            tmp += abs(x1-x2) + abs(y1-y2)
        ans = min(ans, tmp)
    return ans


## 问题50：[删除一次得到子数组最大和](https://leetcode.cn/problems/maximum-subarray-sum-with-one-deletion/description/)
### 题目
给你一个整数数组，返回它的某个 非空 子数组（连续元素）在执行一次可选的删除操作后，所能得到的最大元素总和。换句话说，你可以从原数组中选出一个子数组，并可以决定要不要从中删除一个元素（只能删一次哦），（删除后）子数组中至少应当有一个元素，然后该子数组（剩下）的元素总和是所有子数组之中最大的。注意，删除一个元素后，子数组不能为空。

### 分析
- 这种"可选"删除类型的题目, 多尝试用动态规划。这种含子数组的多尝试用前缀和, 定义右端点的动态规划
- 这题包含有"可选", 子数组类型, 因此尝试使用定义右端点的动态规划(即选与不选)。
- 定义dp[i][0]: 为右端点为i的，并且不删除任何数字的子数组元素和；dp[i][1]: 为右端点为i的，必须删除1个数字的子数组元素和
- 分析问题我们可以知道, 当遍历到i时:
    - dp[i][0] = max(dp[i-1][0] + arr[i], arr[i]) , 即可以将arr[i]并入到之前字符串中, 或重新另起一个子数组
    - dp[i][1] = max(dp[i-1][1] + arr[i], dp[i-1][0]) 即前面已经删除了一个元素+当前元素 或 前面没有删除元素+删除当前元素。
- 初始状态: dp[0][0] = arr[0]; dp[0][1] = 0 (模拟删除当前元素)

In [ ]:
def process(arr):
    n = len(arr)
    dp = [[0, 0] for _ in range(n)]
    dp[0][0] = arr[0]
    dp[0][1] = 0
    ans = arr[0]  # 由于必须要保留一个元素
    for i in range(1, n):
        dp[i][0] = max(dp[i-1][0] + arr[i], arr[i])
        dp[i][1] = max(dp[i-1][1] + arr[i], dp[i-1][0])
        ans = max(ans, dp[i][0], dp[i][1])
    return ans

arr = [2,1,-2,-5,-2]
process(arr)

## 问题51：[引爆最多的炸弹](https://leetcode.cn/problems/detonate-the-maximum-bombs/description/)
### 题目
给你一个炸弹列表。一个炸弹的爆炸范围定义为以炸弹为圆心的一个圆。炸弹用一个下标从0开始的二维整数数组bombs表示，其中bombs[i] = [xi,yi, ri]。xi和yi表示第i个炸弹的X和Y坐标，ri表示爆炸范围的半径。你需要选择引爆一个炸弹。当这个炸弹被引爆时，所有在它爆炸范围内的炸弹都会被引爆，这些炸弹会进一步将它们爆炸范围内的其他炸弹引爆。

给你数组bombs，请你返回在引爆一个炸弹的前提下，最多能引爆的炸弹数目。

### 分析
- 通过坐标以及爆炸范围, 我们可以得到一个单向图, 单向图的联通性我们可以使用记忆化搜索的dfs算法。dfs(root, visit)表示从这个节点出发可以引爆的数量, visit表示已经爆炸的节点(为加快查询我们可以使用visit为[fasle]*n)
- 不可以使用并查集, 因为这是一个单向图

In [ ]:
from collections import defaultdict
import itertools
def process(bombs):
    n = len(bombs)
    graph = defaultdict(list)
    # 得到单向图(我们也可以使用双重循环来得到)
    for (index1, (x1,y1,r1)), (index2, (x2,y2,r2)) in itertools.combinations(enumerate(bombs), 2):
        dis = (x1-x2)**2 + (y1-y2)**2
        if dis <= r1**2:
            graph[index1].append(index2)
        if dis <= r2**2:
            graph[index2].append(index1)

    def dfs(root, visit):
        res = 1
        if root not in graph:  # 说明没有其他炸弹可以被引爆
            return res

        neighbors = graph[root]
        for node in neighbors:
            if visit[node]:  # 若被访问过
                continue
            visit[node] = True
            res += dfs(node, visit)
        return res

    ans = 1
    for root in graph:
        visit = [False] * n
        visit[root] = True
        ans = max(ans, dfs(root, visit))

    return ans

bombs = [[4,4,3],[4,4,3]]
process(bombs)

## 问题52：[求出所有子序列的能量和](https://leetcode.cn/problems/find-the-sum-of-subsequence-powers/description/)
### 题目
子序列的能量定义为: 子序列中任意两个元素的差值绝对值的最小值。请你返回nums中长度等于k的所有子序列的能量和。

### 分析
- 对子序列排序后，其能量为两两相邻元素的差值绝对值的最小值。因此我们可以提前对原数组nums排序后，我们在计算子序列能量时只需考虑相邻元素之间的差值即可。
- 对于子序列问题动态规划常定义dp[i]为以i位置结尾的子序列...。此题目中由于我们需要判断长度k, 因此我们还需要一维度dp[i][p]: 以i结尾长度为p的子序列能量和。以i结尾长度为p的子序列每个子序列的能量都不一样的, 因此我们还需要加一维度表示能量。即最后定义dp[i][p][v]: 以i结尾长度为p的子序列且能量为v的子序列数量。(最终我们直接使用v * dp[i][k][v]对i, v进行累加即可得到最终答案)
- 转移方程: dp[i][p][v]
    - 我们需要遍历j < i的所有j, 计算出diff=abs(nums[i]−nums[j]) 因为j是以j结尾的p-1长度的子序列, 因此可能增加的能量就是diff。
    - 只需枚举所有 d[j][p−1][v]，然后更新(累加)到 d[i][p][min(diff,v)] 即可。因为如果枚举到的v大于diff, 则以diff为能量。如果枚举到的v小于diff，则以v为能量
- 由于最后一维度v是不能提前知道的, 因此我们可以用字典来储存, 从而减少遍历量, 我们定义dp[i][1][inf] = 1: 即以i结尾长度为1的子字符串的能量无穷大, 这样在d[i][2][min(diff,v)]更新时dp[i][2][diff]

In [ ]:
from collections import defaultdict

def process(nums, k):
    nums.sort()
    n = len(nums)
    dp = [ [defaultdict(int) for _ in range(k + 1)] for _ in range(n)]  # 注意这里不能使用[defaultdict(int)] * k+1
    ans = 0

    for i in range(n):
        dp[i][1][float("inf")] = 1

        for p in range(2, k+1):
            for j in range(i):
                diff = abs(nums[i] - nums[j])
                v_dict = dp[j][p-1]
                for v in v_dict:
                    dp[i][p][min(diff, v)] = (v_dict[v] + dp[i][p][min(diff, v)]) % int(1e9 + 7)  # 提前取模
        # 遍历每一个v, cnt
        for v, cnt in dp[i][k].items():
            ans += v * cnt
            ans %= int(1e9 + 7)
    
    return ans
            
nums = [1,2,3,4]
k = 3
process(nums, k)


In [ ]:
def process(nums, k):
    n = len(nums)
    res = 0
    d = [[defaultdict(int) for _ in range(k + 1)] for _ in range(n)]  
    nums.sort()

    for i in range(n):
        d[i][1][float("inf")] = 1
        for p in range(2, k + 1):
            for j in range(i):
                diff = abs(nums[i] - nums[j])
                v_dict = d[j][p-1]
                print(v_dict)
                for v in v_dict:
                    d[i][p][min(diff, v)] = (d[i][p][min(diff, v)] + v_dict[v]) % int(1e9 + 7)

        for v, cnt in d[i][k].items():
            res = (res + v * cnt % int(1e9 + 7)) % int(1e9 + 7)

    return res

nums = [1,2,3,4]
k = 3
process(nums, k)

## 问题53：[生成特殊数字的最少操作](https://leetcode.cn/problems/minimum-operations-to-make-a-special-number/description/)
### 题目
给你一个下标从0开始的字符串 num ，表示一个非负整数。在一次操作中，您可以选择 num 的任意一位数字并将其删除。请注意，如果你删除 num 中的所有数字，则 num 变为 0。返回最少需要多少次操作可以使num变成可以被25整除(0也被视为能被25整除)。

### 分析
- 由于0能被25整除, 因此我们至少可以把num全部删除得到结果，如果num中包含有0, 那么我们可以只用删除n-1个数字即可
- 规律发现能被25整除的数末尾都为: 25, 00, 75, 50。以50举例, 我们可以从右往左遍历, 找到第一个0的位置(假设为i), 接着遍历[0, i]查找最后一个5的位置(假设为j), 那么我们只用删除n-1-j-1个数字, 就可以得到末尾为50了。其他情况也可以如此得到
- 使用str.rfind(sub, start, end): 在str中在范围[start, end)中从右往左找字符串sub的第一次出现的位置, 返回首坐标. 如果不存在则返回-1

In [ ]:
def f(num, tail):
    n = len(num)
    i = num.rfind(tail[1]) # 找到tail的第二个数字
    if i <= 0:    # 如果tail的第二个数字在0位置找到, 那tail的第一个数字一定找不到了, 所以可以直接返回了
        return n   # 意味着要删除所有数字
    
    j = num.rfind(tail[0], 0, i) # 在范围[start, end)从右往左寻找tail的第一个数字
    return n if j < 0 else n-2-j
    
def process(num):
    n = len(num)
    # 至少可以通过删除n个数字或者n-1个数字得到
    ans = n - ("0" in num)   # true代表1
    ans = min(ans, f(num, "00"), f(num, "25"), f(num, "50"), f(num, "75"))
    return ans

num = "2245047"
process(num)

## 问题54：[找出分区值](https://leetcode.cn/problems/find-the-value-of-the-partition/description/)
### 题目
给你一个正整数数组nums 。将 nums 分成两个数组：nums1 和 nums2 ，并满足下述条件：数组 nums 中的每个元素都属于数组 nums1 或数组 nums2 。两个数组都非空 。分区值定义为: 最小的|max(nums1) - min(nums2)| 。请返回表示分区值的整数。

### 分析
- 将排序后的nums, 计算相邻间隔的值即可(因为排序后在任意位置切一刀都能得到两个非空分区)

In [ ]:
import itertools

def process(nums):
    nums.sort()
    ans = float("inf")
    for i, j in itertools.pairwise(nums):
        ans = min(ans, j-i)
    return ans

nums = [1,3,2,4]
process(nums)

## 问题55：[满足距离约束且字典序最小的字符串](https://leetcode.cn/problems/lexicographically-smallest-string-after-operations-with-constraint/description/)
### 题目
定义函数 distance(s1, s2)，用于衡量两个长度为n的字符串s1和s2之间的距离，即：字符'a'到'z'按循环顺序排列，对于区间[0, n-1]中的i，计算所有s1[i]和s2[i]之间最小距离的和。例如: distance("ab", "cd") == 4

给你一个字符串s和一个整数k。你可以对字符串s执行任意次操作。在每次操作中，可以将s中的一个字母改变为任意其他小写英文字母。返回一个字符串t，为s变换后字典序最小的字符串t，且满足distance(s, t) <= k。

### 分析
- distance=abs(ord(a) - ord(b)), min(distance, 26-distance)可以得到两个任意字母a,b间的循环距离
- 由于t是由s变换字母而来, 因此我们需要从左向右尽可能的将字母往a靠拢, 遍历每个s的字母i, 如果i距离"a"的距离小于k则将其变为"a", 同时距离k减少. 如果当前i与"a"的距离会超过限制, 那么可以将i向"a"尽可能靠近(后退k步骤得到结果, 将字典序尽可能变小)


In [ ]:
def f(a, b):  # 单词计算距离
    distance = abs(ord(a) - ord(b))
    return min(distance, 26-distance)

def process(s: str, k: int):
    t = list(s)
    for i, letter in enumerate(s):
        dis = f(letter, "a")
        if dis <= k:  # 说明可以变为a
            t[i] = "a"
            k -= dis
        else:
            t[i] = chr(ord(letter) - k)
            break
    return "".join(t)
            

s = "zbbz"
k = 3
process(s, k)


## 问题56：[覆盖所有点的最少矩形数目](https://leetcode.cn/problems/minimum-rectangles-to-cover-points/description/)
### 题目
给你一个二维整数数组point，其中points[i] = [xi, yi] 表示二维平面内的一个点。同时给你一个整数w 。你需要用矩形覆盖所有点。每个矩形的左下角在某个点 (x1, 0)处，且右上角在某个点(x2, y2)处，其中x1<= x2 且 y2 >= 0 ，同时对于每个矩形都必须满足x2-x1 <= w 。如果一个点在矩形内或者在边上，我们说这个点被矩形覆盖了。请你在确保每个点都至少被一个矩形覆盖的前提下，最少需要多少个矩形。

注意：一个点可以被多个矩形覆盖。
### 分析
- 由于不需要对y进行限制, 因此我们可以使用贪心算法, 先将points根据x从小道道排序, 然后使用end记录下当前矩形最多可以覆盖到的x的长度。遍历整个points的点，如果当前点无法被矩形覆盖到, 那么矩形个数+1并以该点x为起点, 更新矩形的end

In [ ]:
def process(points, w):
    points.sort(key=lambda x: x[0])
    ans = 1
    end = points[0][0] + w
    for x, _ in points:
        if x <= end:
            continue
        else:
            end = x + w
            ans += 1
    return ans

points = [[0,0],[1,1],[2,2],[3,3],[4,4],[5,5],[6,6]]
w = 2
process(points, w)

## 问题57：[心算挑战](https://leetcode.cn/problems/uOAnQW/description/)
### 题目
要求选手从 N 张卡牌中选出 cnt 张卡牌，若这 cnt 张卡牌数字总和为偶数，则选手成绩「有效」且得分为cnt张卡牌数字总和。 给定数组cards 和cnt, 其中cards[i]表示第i张卡牌上的数字。请帮参赛选手计算最大的有效得分。若不存在获取有效得分的卡牌方案，则返回 0。


### 分析
- 为了选取最大的数, 我们可以先排序后选取最大的cnt个数字, 然后判断是否为偶数, 如果是则直接返回, 如果不是那么我们需要进行替换了.
- 为了能使和为偶数, 我们一定要将原本的cnt数字中换出一些数字, 再换进来一些数字, 换出去的数字和换进来的数字和奇偶性一定不同。因此我们只用换1个数字就可以了。我们有两种换法: 1、去掉偶数, 加上奇数; 2、去掉奇数, 加上偶数。因此我们进行以下操作:
    - 去掉最小的偶数, 加上剩余数字中最大的奇数
    - 去掉最小奇数, 加上剩余数字中最大的偶数
    - 然后我们对两种情况求最大值即可

In [ ]:
def process(cards, cnt):
    cards.sort(reverse=True)
    ans = sum(cards[:cnt])
    if ans % 2 == 0:
        return ans

    # x1和x2就是要被替换出去的两个数
    x1 = cards[cnt-1]    # 一定是最小的奇数 / 最小的偶数
    x2 = x1
    for i in cards[cnt-1::-1]:
        if x1%2 != i%2:   # 在cards中找到与x1不同的奇偶性的另一个数
            x2 = i
            break

    # 初始为0
    ans1 = 0
    ans2 = 0
    # 如果找到了可替换的对象则进行替换
    for j in cards[cnt:]:
        if j%2 != x1%2:
            ans1 = ans - x1 + j
            break
    for j in cards[cnt:]:
        if j%2 != x2%2:
            ans2 = ans - x2 + j
            break

    return max(ans1, ans2)

cards = [1,3,4,5]
cnt = 4
process(cards, cnt)

## 问题58：[正方形中的最多点数](https://leetcode.cn/problems/maximum-points-inside-the-square/description/)
### 题目
给你一个二维数组points和一个字符串s，其中points[i]表示第i个点的坐标，s[i]表示第i个点的标签。如果一个正方形的中心在(0, 0)，所有边都平行于坐标轴，且正方形内不存在标签相同的两个点，那么我们称这个正方形是合法的。请你返回合法正方形中可以包含的最多点数。

### 分析
- 由于中心在(0,0), 对于每一个标签坐标(x,y)我们只需要计算max(abs(x), abs(y))与r的大小, 即可判定该点是否在以2*r为边长的正方形中
- 遍历points之后可以计算出所有的点的(d, 标签), 根据d排序, 并维护一个set()表示该正方形中已经存在的点, 由于可能存在(3, a)(3,a)这样的情况, 因此我们以r遍历(在遍历points时记录下最大的r范围), 其中r的最大值是: 所有label的次小距离的最小值

In [ ]:
import bisect
import collections

def process(points, s):
    max_r = float("inf")  # 记录所有label的次小距离的最小值
    label_map = collections.defaultdict(lambda: float("inf"))  # 每个元素的初始值为

    for (i, j), x in zip(points, s):
        d = max(abs(i), abs(j))
        if d < label_map[x]:  # 说明d为当前x的最小值, label_map[x]为次小值
            max_r = min(max_r, label_map[x])
            label_map[x] = d
        else:   # d可能为次小值
            max_r = min(max_r, d)

    ans = 0
    for d in label_map.values():
        if d < max_r:
            ans += 1
    return ans
    
points = [[16,32],[27,3],[23,-14],[-32,-16],[-3,26],[-14,33]]
s = "aaabfc"
process(points, s)

## 问题59：[另一棵树的子树](https://leetcode.cn/problems/subtree-of-another-tree/description/)
### 题目
给你两棵二叉树root和subRoot。检验root中是否包含和subRoot具有相同结构和节点值的子树。如果存在，返回true; 否则，返回 false。二叉树tree的一棵子树包括tree的某个节点和这个节点的所有后代节点。tree 也可以看做它自身的一棵子树。

### 分析
- 我们可以通过遍历得到两个树完全相等: 两个节点的值是相同的, 且判断node1和node2的左右子树是否也是完全相同的(isSame)取且的关系. 
- 我们判断是否存在子树关系只用判断 
    - 当前两棵树相等 
    - 或者，t 是 s 的左子树；
    - 或者，t 是 s 的右子树。
    

In [ ]:
def isSame(node1, node2):
    if node2 == None and node1 == None:
        return True
    if node1 == None or node2 == None:   # 说明node1和node2有一个为空则一定不可能是同一树
        return False
    
    # 当前两个树的根节点值相等；
    # 并且，s的左子树和t的左子树相等;
    # 并且，s的右子树和t的右子树相等。
    return node1.val == node2.val and isSame(node1.left, node2.left) and isSame(node1.right, node2.right)

def isSub(node1, node2):
    if node2 == None and node1 == None:
        return True
    if node1 == None or node2 == None:   # 说明node1和node2有一个为空则一定不可能是同一树
        return False
    
    # 当前两棵树相等；(因为两个树相等那么一定是互为sub的)
    # 或者，t 是 s 的左子树；
    # 或者，t 是 s 的右子树。
    return isSame(node1, node2) or isSub(node1.left, node2) or isSub(node1.right, node2)


## 问题60：[找出所有稳定的二进制数组 I/II](https://leetcode.cn/problems/find-all-possible-stable-binary-arrays-i/description/)
### 题目
给你3个正整数zero，one和limit。一个 二进制数组arr如果满足以下条件，那么我们称它是稳定的：
- 0在arr中出现次数 恰好 为 zero 。
- 1在arr中出现次数 恰好 为 one 。
- arr中每个长度超过limit的子数组都同时包含0和1。

请你返回稳定二进制数组的总数目。

### 分析
- dfs(i,j,k)表示用i个0和j个1构造稳定数组的方案数，其中第i+j个位置要填k，其中k为0或1
- 计算dfs(i,j,0)我们需要得到 dfs(i-1,j,0) 和dfs(i-1,j,1)
    - 但是要注意dfs(i-1,j,0)中包含了「最后连续limit个位置填0」这种情况, 因为...0000也是符合要求dfs(i-1,j,0), 但在此末尾上在加一个0, 就会超标, 与dfs(i,j,0)定义不符合。因此在计算dfs(i,j,0)时不能直接等于dfs(i-1,j,0) + dfs(i-1,j,1)。需要在dfs(i-1,j,0)的情况中减去末尾连续limit个位置填0的情况数
    - 如何计算末尾连续limit个位置填0的情况数。因为dfs的定义是稳定数组的方案数，只包含合法方案，所以在最后连续limit个位置填0的情况下，倒数第limit+1个位置一定要填1，这有dfs(i−limit−1, j, 1)种方案。也就是说只有这种情况下，才能支持末为连续为0(其实也就是...0000这种情况中...的情况数)
    - 最终dfs(i,j,0) = dfs(i-1,j,1) + dfs(i-1,j,0) - dfs(i−limit−1,j,1)
    - 同理dfs(i,j,1) = dfs(i,j-1,0) + dfs(i,j-1,1) - dfs(i,j-limit-1,0)

In [ ]:
import functools

@functools.cache
def dfs(i, j, k, limit):
    if i == 0:
        return 1 if j<=limit and k==1 else 0  # 当需要以1结尾且j小于limit时, 只有一种情况
    if j == 0:
        return 1 if i<=limit and k==0 else 0  # 当需要以0结尾且i小于limit时, 只有一种情况
    
    if k == 0:
        ans = dfs(i-1,j,1, limit) + dfs(i-1,j,0,limit)
        ans -= dfs(i-limit-1, j, 1, limit) if i-limit-1 >= 0 else 0
        return ans
    if k == 1:
        ans = dfs(i,j-1,0,limit) + dfs(i,j-1,1,limit)
        ans -= dfs(i,j-limit-1,0,limit) if j-limit-1 >= 0 else 0
        return ans
    
def process(zero: int, one: int, limit: int):
    ans = (dfs(zero, one, 0, limit) + dfs(zero, one, 1, limit)) % (1e9+7)
    dfs.cache_clear()
    return int(ans)

zero = 1
one = 1
limit = 2
process(zero,one,limit)


## 问题61：[找出与数组相加的整数 II](https://leetcode.cn/problems/find-the-integer-added-to-array-ii/description/)
### 题目
给你两个整数数组nums1和nums2。从nums1中移除两个元素，并且所有其他元素都与变量x所表示的整数相加。如果x为负数，则表现为元素值的减少。执行上述操作后，nums1和nums2相等。当两个数组中包含相同的整数，并且这些整数出现的频次相同时，两个数组相等。

返回能够实现数组相等的最小整数x。

### 分析
- 根据[找出与数组相加的整数 I]我们可以知道当移除nums1的两个数字后, 计算x的方式依然是min(nums2) - min(nums1)。因此我们可以得知nums1的前三小的值至少有一个会被留下来(因此只移除nums1的两个数字)
- 我们可以将nums1和nums2排序后, 枚举留下的三种情况, 分别是: 留下的是nums1[2], nums1[1], nums1[0],比如留下的是nums1[1], 那么可以计算得到x = min(nums2) - nums1[1]. 这个时候我们将nums1的所有元素都+x, 此时, 如果x为答案, 那么nums2一定是nums1的子序列. 如果是则说明此时的x就是答案
- 由于需要返回最小的x, 因此nums1留下的应该越大越大好(有可能nums1[2], nums1[1], nums1[0]留下任何一个都满足条件)。因此我们在枚举时用可以先枚举nums[2], 如果满足条件则直接返回; 再判断留下的是否为nums[1], 最后判断nums1[0]
- 我们可以使用同向双指针(需要排序后)来判断子序列


In [5]:
# 判断nums2是否为nums1的子序列
def isSubSqu(nums1, nums2):
    i, j = 0, 0
    n = len(nums1)
    m = len(nums2)
    while i < n:
        if nums1[i] == nums2[j]:
            j += 1
            if j == m:  # 说明nums2的所有数字都找完了
                return True
        i += 1
    # 当nums1已经到了终点还没匹配完nums2, 说明nums2不是子序列
    return False
    

def process(nums1, nums2):
    nums1.sort()
    nums2.sort()
    x = nums2[0] - nums1[2]
    if isSubSqu([i+x for i in nums1], nums2):
        return x

    x = nums2[0] - nums1[1]
    if isSubSqu([i+x for i in nums1], nums2):
        return x
    
    return nums2[0] - nums1[0]

nums1 = [4,6,3,1,4,2,10,9,5]
nums2 = [5,10,3,2,6,1,9]

process(nums1, nums2)

0

## 问题62：[实现一个魔法字典](https://leetcode.cn/problems/implement-magic-dictionary/description/)
### 题目
设计一个使用单词列表进行初始化的数据结构，单词列表中的单词互不相同。 如果给出一个单词，请判定能否只将这个单词中一个字母换成另一个字母，使得所形成的新单词存在于你构建的字典中。如果可以则返回true, 否则返回fasle

你需要实现__init__, buildDict(self, dictionary: List[str]), search(self, searchWord: str)三个方法

### 分析
- 我们可以先根据字符长度将word进行分组, 每次查询时只用判断相同长度的元素是否相同即可


In [ ]:
from collections import defaultdict
class MagicDictionary:

    def __init__(self):
        self.d = defaultdict(list)

    def buildDict(self, dictionary: List[str]) -> None:
        for word in dictionary:
            self.d[len(word)].append(word)

    # s和d是否只相差1
    def match(self, s, d):
        diff = 0
        for i, j in zip(s, d):
            diff += (i != j)   # 如果i!=j则diff+1
            if diff > 1:
                return False
        if diff == 1:
            return True
        return False

    def search(self, searchWord: str) -> bool:
        for word in self.d[len(searchWord)]:
            if self.match(word, searchWord):
                return True
        return False

## 问题63：[特殊数组 II](https://leetcode.cn/problems/special-array-ii/description/)
### 题目
如果数组的每一对相邻元素都是两个奇偶性不同的数字，则该数组被认为是一个特殊数组。你有一个整数数组nums和一个二维整数矩阵queries，对于 queries[i] = [fromi, toi]，请你帮助你检查子数组nums[fromi:toi+1] 是不是一个特殊数组 。返回布尔数组answer, 如果nums[fromi:toi+1]是特殊数组, 则answer[i]为true, 否则为answer[i]为false。

### 分析
- 使用前缀和: first[i]表示[0,i]这个子数组中有多少个不符合特殊数组要求的数字. nums[fromi:toi+1]是否为特殊数组, 只用考察first[toi] -first[fromi]是否为0即可
- 方法二: 定义长为n−1的数组a, 如果nums[i]与nums[i+1]是同奇偶的, 那么a[i]=0, 最终我们得到的结果是a = [0,1,0,0,1,0,1...], 据此我们可以通过前缀和的方式, 如果first[toi] - first[fromi] = 0 则说明a[fromi:toi+1]中所有数都是0, 因此可以说明nums[fromi:toi+1]是否为特殊数组。其实我们通过这种方式将nums映射为了一个0,1 数组, 而判断0,1数组的子数组是否全为0的方法就是前缀和
- python中我们可以使用functools.accumulate(可迭代对象, initial=0)很容易的得到可迭代对象的前缀和, 这里的initial=0表示第一个元素是与0相加

In [5]:
def process(nums, queries):
    n = len(nums)
    first = [0] * n
    for i in range(1, n):
        if nums[i]%2 == nums[i-1]%2:  # 说明i位置与前面是违背的
            first[i] = first[i-1] + 1
        else:
            first[i] = first[i-1]
    
    ans = [True] * len(queries)
    for index, (from_i, to_i) in enumerate(queries):
        if first[to_i] - first[from_i] != 0:
            ans[index] = False
    return ans

nums = [4,3,1,6]
queries = [[0,2],[2,3]]
process(nums, queries)

[False, True]

## 问题64：[学生出勤记录 II](https://leetcode.cn/problems/student-attendance-record-ii/description/)
### 题目
可以用字符串表示一个学生的出勤记录，其中的每个字符用来标记当天的出勤情况（缺勤、迟到、到场）。记录中只含下面三种字符：
- 'A'：Absent，缺勤
- 'L'：Late，迟到
- 'P'：Present，到场
如果学生能够同时满足下面('A'的个数 < 2) 和(不存在连续3天或连续3天以上的迟到'L'记录)，则可以获得出勤奖励。给你一个整数n，表示出勤记录的次数。请你返回记录长度为n时，可能获得出勤奖励的记录情况数量。

### 分析
- 动态规划: 当需要进行各种情况排序时, 即每个位置是有限的情况数时, 那在每个位置选择时就是一个选与不选的情况, 因此使用动态规划。
- 按顺序填入时我们需要知道:
    - 还剩下多少个字母需要填。
    - 已经填了多少个A。如果之前填过A，那么后续不能填A。
    - 相邻位置上有多少个连续L。如果之前连续填了2个L，那么当前位置不能填L。
- dp[i][j][k]: 定义为长度为i最后一个位置, 一共填了j个A, 且连续填了k个L的情况数. 其中j=0或者1, k=0,1,2
- dp[i][j][k]的结果时由: 
    - 当k >= 1 时, dp[i][j][k] = dp[i-1][j][k-1]  (只能由i-1位置加一个L得到)
    - 当k == 0 时, dp[i][j][k] = sum(dp[i-1][j]) + sum(dp[i-1][j-1])      (可以在i-1情况任意k情况下加一个P或者在i-1情况任意k情况加一个A得到)
- 初始条件: dp[1][0][0] = 1, dp[1][1][0] = 1, dp[1][0][1] = 1


In [6]:
def process(n):
    MOD = 1_000_000_007
    dp = [ [[0]*3, [0]*3] for _ in range(n+1)]
    dp[1][0][0] = 1 # P
    dp[1][1][0] = 1 # A
    dp[1][0][1] = 1 # L
    for i in range(2, n+1):
        for j in range(2):
            for k in range(3):
                if k >= 1:
                    dp[i][j][k] = dp[i-1][j][k-1] % MOD
                else:
                    dp[i][j][k] = (sum(dp[i-1][j]) + (sum(dp[i-1][j-1]) if j >= 1 else 0)) % MOD
                    
    return (sum(dp[n][1]) + sum(dp[n][0])) % MOD

n = 10101
process(n)

[[[0, 0, 0], [0, 0, 0]], [[1, 1, 0], [1, 0, 0]], [[2, 1, 1], [3, 1, 0]], [[4, 2, 1], [8, 3, 1]], [[7, 4, 2], [19, 8, 3]], [[13, 7, 4], [43, 19, 8]], [[24, 13, 7], [94, 43, 19]], [[44, 24, 13], [200, 94, 43]], [[81, 44, 24], [418, 200, 94]], [[149, 81, 44], [861, 418, 200]], [[274, 149, 81], [1753, 861, 418]], [[504, 274, 149], [3536, 1753, 861]], [[927, 504, 274], [7077, 3536, 1753]], [[1705, 927, 504], [14071, 7077, 3536]], [[3136, 1705, 927], [27820, 14071, 7077]], [[5768, 3136, 1705], [54736, 27820, 14071]], [[10609, 5768, 3136], [107236, 54736, 27820]], [[19513, 10609, 5768], [209305, 107236, 54736]], [[35890, 19513, 10609], [407167, 209305, 107236]], [[66012, 35890, 19513], [789720, 407167, 209305]], [[121415, 66012, 35890], [1527607, 789720, 407167]], [[223317, 121415, 66012], [2947811, 1527607, 789720]], [[410744, 223317, 121415], [5675882, 2947811, 1527607]], [[755476, 410744, 223317], [10906776, 5675882, 2947811]], [[1389537, 755476, 410744], [20920006, 10906776, 5675882]], [[

183236316

## 问题65：[统计特殊整数（数位DP）](https://leetcode.cn/problems/count-special-integers/description/)
### 题目
如果一个正整数每一个数位都是互不相同的，我们称它是特殊整数 。给你一个正整数n，请你返回区间[1, n]之间特殊整数的数目。

### 分析
- 数位DP的思想: 我们需要在指定位数上填写数字, 每个数位可以填多个数字, 填完这些数字得到的整数需要满足某些条件, 常常问这样填数字的方案有多少?或者在满足某些情况时停止
- 数位DP的模板代码: 定义f(i, mask, isLimit, isNum) 表示构造第i位及其之后数位的合法方案数(i=0表示填写第0位及其以后数字的个数)。
    - mask: 由于此题要求每一个数字都不同因此我们使用mask表示已经选取了哪些数字. 
    - isLimit: 表示当前位是否有限制（true表示当前为需要有上界限制）, 比如在本题中假设我们的值要小于123, 如果当前面数位的值刚好为1 2 _, 那么我们填写第三位时只能填写[0, 3]范围, 但如果前面的值为1 0 _, 那么第三位的值可以是[0, 9]任意数, 因此我们需要用这个表示当前数位可以取哪些范围
    - isNum: 表示在之前的位置上是否已经填了数字了（true表示前面已经有数字了）, 我们需要用这个标识来帮我们确定, 当前数位是否可以填0, 如果前面并没有填写数字, 那么我们这一数位是不能从0开始填的。这个标识也可以帮助我们计算 小于n的位数的情况
- 对于递归我们需要使用@cache进行记忆化缓存, 因此我们的参数中不能含有集合这种元素, 哪如何去表示mask呢? 实际上数字集合与二进制数字是一一对应的, 00010: 表示数字1已经使用, 01001:表示数字1和数字3已经使用
    - 往集合中填一个数字d: 将 mask 更新为 mask | (1 << d)
    - 判断集合中是否含有数字d: mask >> d & 1 可以取出mask的第d个比特位，如果是1就说明d在集合中。
    - 往集合中删除一个数字d: 将mask 更新为 mask ^ (1 << d)


In [ ]:
from functools import cache

def process(n):
    s = str(n)  # 将n字符串化方便统计位数

    @cache
    def f(i, mask, isLimit, isNum):
        if i == len(s):   # 如果i已经到了最末尾
            # 返回1或者0, 如果isNum=Fasle, 意味着前面都没有数字, 因此返回0, 否则返回1
            return int(isNum)  # 等价于 1 if isNum else 0

        res = 0  # 用于统计第i位置及其之后数位的合法方案数
        if isNum == False: # 表示第i位以前没有数字, 则i位置也可以选择跳过(需要加上这种情况)
            res += f(i+1, mask, False, False)     # 由于当前位选择了跳过, 则对下一位一定是没有限制的

        # 根据isLimit计算遍历的上界
        up = int(s[i]) if isLimit else 9    # 如果有限制, 则本位最大能选择s[i]
        # 根据isNum计算下界
        down = 0 if isNum else 1   # 如果前文已经有了数字了, 那么本位可以从0填写

        for d in range(down, up+1):
            if mask >> d & 1 == 0:  # 如果mask中没有d
                res += f(i+1, mask | (1 << d), isLimit and d == up, True)  # 如有要对后续数位有上界限制, 必须是之前所有数位都达到限制
        
        return res
    # 初始条件中需要使用isLimit=True是因为我们需要对第0位有上界限制
    return f(0, 0, True, False)


## 问题66：[]()
### 题目

### 分析

## 问题67：[]()
### 题目

### 分析

## 问题68：[]()
### 题目

### 分析